<div class="alert alert-block alert-success">

<!-- -info-warning-success-danger>   --> <font size=5>
<b>
    17 июля 2023<br>
    расчет выдачи и возврата по пациентам (!!! ТОЛЬКО ДЛЯ НОМЕНКЛАТУР, с УКАЗАННЫМ инвентарным номером)<br>
    </b>    </div>
<hr>

In [849]:
import pandas as pd
import numpy as np
from IPython.display import display_markdown
from datetime import datetime, date
import warnings
warnings.filterwarnings('ignore')

####
<div class="alert alert-block alert-success">
    <!-- -info-warning-success-danger>   -->
<font size=5>
<b>подключаем выдачу
    </b>    </div>
<hr>

In [850]:
data_poluchenie = pd.read_csv('выдача_1с_все_pand.csv', delimiter=';', encoding='cp1251')
print(data_poluchenie.shape)

(21592, 7)


<div class="alert alert-block alert-info">

<!-- <div class="alert alert-block alert-info">
<div class="alert alert-block alert-warning">
<div class="alert alert-block alert-success">
<div class="alert alert-block alert-danger">
     -->
<font size=3>
<b>
    Убираем лишние колонки  и переименовываем
    </b>
    </div>
<hr>

In [851]:
# ---------------------- функция преобразования к нижнему РЕГИСТРУ
def def_low(df):
    for i in df.columns:
        df[i] =  df[i].str.lower()
    return df    

In [852]:
# как4ие есть колонки
# data_poluchenie.columns

In [853]:
# уберем лишние колонки
df_poluchenie_nomenk = data_poluchenie.copy()
df_poluchenie_nomenk = df_poluchenie_nomenk[['Пациент.ФИО', 'Пациент.Дата рождения', 'Пациент.Пол', 'Дата выдачи',
       'Оборудование.Номенклатура', 'Оборудование.Инвентарный номер',
                ]]
# df_poluchenie_nomenk

In [854]:
#  переименуем покороче столбцы
df_poluchenie_nomenk.rename(columns={'Пациент.ФИО': 'Пациент_получ'}, inplace=True)
df_poluchenie_nomenk.rename(columns={'Пациент.Дата рождения': 'ДР_получ'}, inplace=True)
df_poluchenie_nomenk.rename(columns={'Пациент.Пол': 'Пол_получ'}, inplace=True)
df_poluchenie_nomenk.rename(columns={'Оборудование.Номенклатура': 'Номенклатура_получ'}, inplace=True)
df_poluchenie_nomenk.rename(columns={'Оборудование.Инвентарный номер': 'Инвент_ном_получ'}, inplace=True)
df_poluchenie_nomenk.rename(columns={'Дата выдачи': 'Дата_выдачи_получ'}, inplace=True)
df_poluchenie_nomenk.head(2)

,Пациент_получ,ДР_получ,Пол_получ,Дата_выдачи_получ,Номенклатура_получ,Инвент_ном_получ
0,Шваков Макар Олегович,14.04.2014,Мужской,14.04.2021,Концентратор кислородный JAY-5A.,004410124-0003072
1,Мартынов Павел Дмитриевич,20.06.2010,Мужской,17.03.2021,Аппарат для удаления выделений из дыхательных ...,004410124-0002667


In [855]:
#  -----------------преобразования к нижнему РЕГИСТРУ
df_poluchenie_nomenk = def_low(df_poluchenie_nomenk)

<div class="alert alert-block alert-info">

<!-- <div class="alert alert-block alert-info">
<div class="alert alert-block alert-warning">
<div class="alert alert-block alert-success">
<div class="alert alert-block alert-danger">
     -->
<font size=3>
<b> Приведем ДАТЫ к фрмату ДАТА</b><br>
            </div>

In [856]:
# ----------------форматнем дату к дате
df_poluchenie_nomenk['ДР_получ'] = pd.to_datetime(df_poluchenie_nomenk['ДР_получ'], dayfirst=True, errors = 'coerce').dt.normalize()
df_poluchenie_nomenk['Дата_выдачи_получ'] = pd.to_datetime(df_poluchenie_nomenk['Дата_выдачи_получ'], dayfirst=True, errors = 'coerce').dt.normalize()
# df_poluchenie_nomenk

<div class="alert alert-block alert-info"><font size=3>
Ключ  старый         </div>

In [857]:
#  ----- Добавим УНИКАЛЬНЫЙ ключ ФИО+ДР - без пробелов ------------
df_poluchenie_nomenk['Ключ'] = df_poluchenie_nomenk['Пациент_получ'].map(str) + (df_poluchenie_nomenk['ДР_получ'].dt.date).map(str) + df_poluchenie_nomenk['Инвент_ном_получ'].map(str)
df_poluchenie_nomenk['Ключ'] = df_poluchenie_nomenk['Ключ'].str.lower()            # ---- все к нижнему регистру
df_poluchenie_nomenk['Ключ'] = df_poluchenie_nomenk['Ключ'].str.replace(' ', '')   # ---- пробелы долой 
# df_poluchenie_nomenk

<div class="alert alert-block alert-info"><font size=3>
Ключ   ПАЦИЕНТ - ПРИБОР         </div>

In [858]:
# ------------------ Новый ключ ПАЦИЕНТ - ПРИБОР --------------
df_poluchenie_nomenk.reset_index(drop= True , inplace= True )
df_poluchenie_nomenk.loc[df_poluchenie_nomenk['Инвент_ном_получ'].str.contains('-') == False, 'Ключ_пац_прибор' ] = df_poluchenie_nomenk['Пациент_получ'].str.split(pat=' ', expand=True)[0] + \
                                                                                                                     '#' + (df_poluchenie_nomenk['ДР_получ'].dt.date).map(str) + \
                                                                                                                     '#' + df_poluchenie_nomenk['Инвент_ном_получ'] + \
                                                                                                                     '#' + df_poluchenie_nomenk['Номенклатура_получ'] 

df_poluchenie_nomenk.loc[df_poluchenie_nomenk['Инвент_ном_получ'].str.contains('-') == True, 'Ключ_пац_прибор' ] = df_poluchenie_nomenk['Пациент_получ'].str.split(pat=' ', expand=True)[0] + \
                                                                                                                    '#' + (df_poluchenie_nomenk['ДР_получ'].dt.date).map(str) + \
                                                                                                                    '#' + df_poluchenie_nomenk['Инвент_ном_получ'] 

<div class="alert alert-block alert-info"><font size=3>
Ключ УДАЛЕНИЯ ПОВТОРОВ ВЫДАЧИ того же тому же в туже дату c номером прибора(если есть) или с названием, если номера нет            </div>

In [859]:
# ------------------ Новый ключ Ключ_3 ФИО НОМЕР (если цифровой) Дата выдачи --------------
df_poluchenie_nomenk.reset_index(drop= True , inplace= True )
df_poluchenie_nomenk.loc[df_poluchenie_nomenk['Инвент_ном_получ'].str.contains('-') == False, 'Ключ_удлн_повт_выдачи' ] = df_poluchenie_nomenk['Пациент_получ'].str.split(pat=' ', expand=True)[0] + \
                                                                                                                     '#' + (df_poluchenie_nomenk['ДР_получ'].dt.date).map(str) + \
                                                                                                                     '#' + df_poluchenie_nomenk['Инвент_ном_получ'] + \
                                                                                                                     '#' + df_poluchenie_nomenk['Номенклатура_получ'] + '#' +  (df_poluchenie_nomenk['Дата_выдачи_получ'].dt.date).map(str)

df_poluchenie_nomenk.loc[df_poluchenie_nomenk['Инвент_ном_получ'].str.contains('-') == True, 'Ключ_удлн_повт_выдачи' ] = df_poluchenie_nomenk['Пациент_получ'].str.split(pat=' ', expand=True)[0] + \
                                                                                                                    '#' + (df_poluchenie_nomenk['ДР_получ'].dt.date).map(str) + \
                                                                                                                    '#' + df_poluchenie_nomenk['Инвент_ном_получ'] + '#' +  (df_poluchenie_nomenk['Дата_выдачи_получ'].dt.date).map(str)

In [860]:
df_poluchenie_nomenk

,Пациент_получ,ДР_получ,Пол_получ,Дата_выдачи_получ,Номенклатура_получ,Инвент_ном_получ,Ключ,Ключ_пац_прибор,Ключ_удлн_повт_выдачи
0,шваков макар олегович,2014-04-14,мужской,2021-04-14,концентратор кислородный jay-5a.,004410124-0003072,шваковмакаролегович2014-04-14004410124-0003072,шваков#2014-04-14#004410124-0003072,шваков#2014-04-14#004410124-0003072#2021-04-14
1,мартынов павел дмитриевич,2010-06-20,мужской,2021-03-17,аппарат для удаления выделений из дыхательных ...,004410124-0002667,мартыновпавелдмитриевич2010-06-20004410124-000...,мартынов#2010-06-20#004410124-0002667,мартынов#2010-06-20#004410124-0002667#2021-03-17
2,дарчиева кристина арсеновна,2010-03-15,женский,2021-03-26,аппарат искуственной вентиляции легких puritan...,004410124-0003302,дарчиевакристинаарсеновна2010-03-15004410124-0...,дарчиева#2010-03-15#004410124-0003302,дарчиева#2010-03-15#004410124-0003302#2021-03-26
3,бабаев асланхон акмалхонович,2019-06-05,мужской,2021-03-31,концентратор кислородный jay-5a.,004410124-0003073,бабаевасланхонакмалхонович2019-06-05004410124-...,бабаев#2019-06-05#004410124-0003073,бабаев#2019-06-05#004410124-0003073#2021-03-31
4,горячев владислав сергеевич,2006-12-28,мужской,2021-04-15,аппарат нивл wm 100 prisma30st,004410124-001791,горячеввладиславсергеевич2006-12-28004410124-0...,горячев#2006-12-28#004410124-001791,горячев#2006-12-28#004410124-001791#2021-04-15
...,...,...,...,...,...,...,...,...,...
21587,задкова есения евгеньевна,2016-03-09,женский,2023-07-17,"отсасыватель хирургический электрический ""arme...",004410134-0083438,задковаесенияевгеньевна2016-03-09004410134-008...,задкова#2016-03-09#004410134-0083438,задкова#2016-03-09#004410134-0083438#2023-07-17
21588,задкова есения евгеньевна,2016-03-09,женский,2023-07-17,"отсасыватель хирургический электрический ""arme...",004410134-0083438,задковаесенияевгеньевна2016-03-09004410134-008...,задкова#2016-03-09#004410134-0083438,задкова#2016-03-09#004410134-0083438#2023-07-17
21589,задкова есения евгеньевна,2016-03-09,женский,2023-07-17,"отсасыватель хирургический электрический ""arme...",004410134-0083438,задковаесенияевгеньевна2016-03-09004410134-008...,задкова#2016-03-09#004410134-0083438,задкова#2016-03-09#004410134-0083438#2023-07-17
21590,задкова есения евгеньевна,2016-03-09,женский,2023-07-17,"отсасыватель хирургический электрический ""arme...",004410134-0083438,задковаесенияевгеньевна2016-03-09004410134-008...,задкова#2016-03-09#004410134-0083438,задкова#2016-03-09#004410134-0083438#2023-07-17


<div class="alert alert-block alert-info"><font size=3>
Удалить <b> ПОВТОРЫ выдач пациент-дата выдачи - прибор

In [861]:
df_poluchenie_nomenk.shape

(21592, 9)

In [862]:
df_poluchenie_nomenk = df_poluchenie_nomenk.drop_duplicates('Ключ_удлн_повт_выдачи')
df_poluchenie_nomenk.shape

(3035, 9)

<div class="alert alert-block alert-info"><font size=3>
УБЕРЕМ <b> НЕПОНЯТНЫЕ ВЫДАЧИ Номенклатуры!!!! 

In [863]:
# ---------------- УБЕРЕМ НЕПОНЯТНЫЕ ВЫДАЧИ!!!!
df_poluchenie_nomenk = df_poluchenie_nomenk.loc[~df_poluchenie_nomenk['Номенклатура_получ'].isna()]

####
<div class="alert alert-block alert-success">
    <!-- -info-warning-success-danger>   -->
<font size=5>
<b>подключаем возврат
    </b>    </div>
<hr>

In [864]:
data_vozvrat = pd.read_csv('возврат_1С_все_pand.csv', delimiter=';', encoding='cp1251')
print(data_vozvrat.shape)

(826, 5)


In [865]:
data_vozvrat = def_low(data_vozvrat)
data_vozvrat

,Номенклатура,Инвентарный номер,Ссылка.Дата,Ссылка.Пациент,Ссылка.Пациент.Дата рождения
0,аппарат искуственной вентиляции легких puritan...,004410124-0003335,11.08.2021 12:00:00,петайкин иван борисович,NaN
1,концентратор кислородный jay-5a.,004410124-0003081,11.08.2021 12:00:00,петайкин иван борисович,NaN
2,аппарат нивл wm 100 prisma30st,004410124-001815,23.06.2021 13:48:15,дыров александр иванович,11.08.2015
3,аппарат искуственной вентиляции легких puritan...,004410124-0003266,15.12.2022 13:26:07,чиркова ангелина сергеевна,02.01.2021
4,аппарат нивл wm 100 prisma30st,004410124-001812,26.05.2022 15:38:23,галан ринат андреевич,20.04.2007
...,...,...,...,...,...
821,"концентратор кислорода invacare, вариант испол...",004410124-001252,23.06.2023 18:14:06,боровков владимир иванович,14.12.1952
822,концентратор кислородный jay в следующем вариа...,004410134-0084495,10.07.2023 17:59:54,баренбойм елена викторовна,02.04.1971
823,концентратор кислородный jay-10,004410124-0004299,26.06.2023 12:17:58,подгорный александр александрович,24.11.1962
824,концентратор кислородный jay в следующем вариа...,004410134-0084459,28.06.2023 10:32:48,обухова ольга ивановна,03.11.1946


<div class="alert alert-block alert-info">
<font size=3><b>
    Убираем лишние колонки и переименовываем 

In [866]:
# уберем лишние колонки
data_vozvrat = data_vozvrat[[
    'Ссылка.Пациент', 'Ссылка.Пациент.Дата рождения',     'Номенклатура', 'Инвентарный номер',       'Ссылка.Дата',
                ]]
#  переименуем покороче столбцы
data_vozvrat.rename(columns={'Ссылка.Пациент': 'Пациент_возвр'}, inplace=True)
data_vozvrat.rename(columns={'Ссылка.Пациент.Дата рождения': 'ДР_возвр'}, inplace=True)
data_vozvrat.rename(columns={'Номенклатура': 'Номенклатура_возвр'}, inplace=True)
data_vozvrat.rename(columns={'Инвентарный номер': 'Инвент_ном_возвр'}, inplace=True)
data_vozvrat.rename(columns={'Ссылка.Дата': 'Дата_возвр'}, inplace=True)
data_vozvrat.head(5)

,Пациент_возвр,ДР_возвр,Номенклатура_возвр,Инвент_ном_возвр,Дата_возвр
0,петайкин иван борисович,NaN,аппарат искуственной вентиляции легких puritan...,004410124-0003335,11.08.2021 12:00:00
1,петайкин иван борисович,NaN,концентратор кислородный jay-5a.,004410124-0003081,11.08.2021 12:00:00
2,дыров александр иванович,11.08.2015,аппарат нивл wm 100 prisma30st,004410124-001815,23.06.2021 13:48:15
3,чиркова ангелина сергеевна,02.01.2021,аппарат искуственной вентиляции легких puritan...,004410124-0003266,15.12.2022 13:26:07
4,галан ринат андреевич,20.04.2007,аппарат нивл wm 100 prisma30st,004410124-001812,26.05.2022 15:38:23


<div class="alert alert-block alert-info">

<!-- <div class="alert alert-block alert-info">
<div class="alert alert-block alert-warning">
<div class="alert alert-block alert-success">
<div class="alert alert-block alert-danger">
     -->
<font size=3>
<b> Приведем ДАТЫ к фрмату ДАТА</b><br>
            </div>

In [867]:
# ----------------форматнем дату к дате
data_vozvrat['ДР_возвр'] = pd.to_datetime(data_vozvrat['ДР_возвр'], dayfirst=True, errors = 'coerce').dt.normalize()
data_vozvrat['Дата_возвр'] = pd.to_datetime(data_vozvrat['Дата_возвр'], dayfirst=True, errors = 'coerce').dt.normalize()
# data_vozvrat

<div class="alert alert-block alert-info"><font size=3>
Ключ            </div>

In [868]:
#  ----- Добавим УНИКАЛЬНЫЙ ключ ФИО+ДР - без пробелов ------------
data_vozvrat['Ключ'] = data_vozvrat['Пациент_возвр'].map(str) + (data_vozvrat['ДР_возвр'].dt.date).map(str) + data_vozvrat['Инвент_ном_возвр'].map(str)
data_vozvrat['Ключ'] = data_vozvrat['Ключ'].str.lower()            # ---- все к нижнему регистру
data_vozvrat['Ключ'] = data_vozvrat['Ключ'].str.replace(' ', '')   # ---- пробелы долой 
data_vozvrat

,Пациент_возвр,ДР_возвр,Номенклатура_возвр,Инвент_ном_возвр,Дата_возвр,Ключ
0,петайкин иван борисович,NaT,аппарат искуственной вентиляции легких puritan...,004410124-0003335,2021-08-11,петайкиниванборисовичnat004410124-0003335
1,петайкин иван борисович,NaT,концентратор кислородный jay-5a.,004410124-0003081,2021-08-11,петайкиниванборисовичnat004410124-0003081
2,дыров александр иванович,2015-08-11,аппарат нивл wm 100 prisma30st,004410124-001815,2021-06-23,дыровалександриванович2015-08-11004410124-001815
3,чиркова ангелина сергеевна,2021-01-02,аппарат искуственной вентиляции легких puritan...,004410124-0003266,2022-12-15,чирковаангелинасергеевна2021-01-02004410124-00...
4,галан ринат андреевич,2007-04-20,аппарат нивл wm 100 prisma30st,004410124-001812,2022-05-26,галанринатандреевич2007-04-20004410124-001812
...,...,...,...,...,...,...
821,боровков владимир иванович,1952-12-14,"концентратор кислорода invacare, вариант испол...",004410124-001252,2023-06-23,боровковвладимириванович1952-12-14004410124-00...
822,баренбойм елена викторовна,1971-04-02,концентратор кислородный jay в следующем вариа...,004410134-0084495,2023-07-10,баренбоймеленавикторовна1971-04-02004410134-00...
823,подгорный александр александрович,1962-11-24,концентратор кислородный jay-10,004410124-0004299,2023-06-26,подгорныйалександралександрович1962-11-2400441...
824,обухова ольга ивановна,1946-11-03,концентратор кислородный jay в следующем вариа...,004410134-0084459,2023-06-28,обуховаольгаивановна1946-11-03004410134-0084459


<div class="alert alert-block alert-info"><font size=3>
Ключ c номером прибора(если есть) или с названием, если номера нет            </div>

In [869]:
# ------------------ Новый ключ Ключ_3 ФИО НОМЕР (если цифровой) Дата выдачи --------------
data_vozvrat.reset_index(drop= True , inplace= True )
data_vozvrat.loc[data_vozvrat['Инвент_ном_возвр'].str.contains('-') == False, 'Ключ_удлн_повт_возвр' ] = data_vozvrat['Пациент_возвр'].str.split(pat=' ', expand=True)[0] + '#' + (data_vozvrat['ДР_возвр'].dt.date).map(str) + \
                                                                                                   '#' + data_vozvrat['Инвент_ном_возвр'] + \
                                                                                                   '#' + data_vozvrat['Номенклатура_возвр'] + '#' + (data_vozvrat['Дата_возвр'].dt.date).map(str)

data_vozvrat.loc[data_vozvrat['Инвент_ном_возвр'].str.contains('-') == True, 'Ключ_удлн_повт_возвр' ] = data_vozvrat['Пациент_возвр'].str.split(pat=' ', expand=True)[0] + \
                                                                                                 '#' + (data_vozvrat['ДР_возвр'].dt.date).map(str) + '#' + data_vozvrat['Инвент_ном_возвр'] + '#' + (data_vozvrat['Дата_возвр'].dt.date).map(str)

<div class="alert alert-block alert-info"><font size=3>
Ключ   ПАЦИЕНТ - ПРИБОР         </div>

In [870]:
# ------------------ Новый ключ ПАЦИЕНТ - ПРИБОР --------------
data_vozvrat.reset_index(drop= True , inplace= True )
data_vozvrat.loc[data_vozvrat['Инвент_ном_возвр'].str.contains('-') == False, 'Ключ_пац_прибор' ] = data_vozvrat['Пациент_возвр'].str.split(pat=' ', expand=True)[0] + \
                                                                                                                     '#' + (data_vozvrat['ДР_возвр'].dt.date).map(str) + \
                                                                                                                     '#' + data_vozvrat['Инвент_ном_возвр'] + \
                                                                                                                     '#' + data_vozvrat['Номенклатура_возвр'] 

data_vozvrat.loc[data_vozvrat['Инвент_ном_возвр'].str.contains('-') == True, 'Ключ_пац_прибор' ] = data_vozvrat['Пациент_возвр'].str.split(pat=' ', expand=True)[0] + \
                                                                                                                    '#' + (data_vozvrat['ДР_возвр'].dt.date).map(str) + \
                                                                                                                    '#' + data_vozvrat['Инвент_ном_возвр'] 

In [871]:
data_vozvrat

,Пациент_возвр,ДР_возвр,Номенклатура_возвр,Инвент_ном_возвр,Дата_возвр,Ключ,Ключ_удлн_повт_возвр,Ключ_пац_прибор
0,петайкин иван борисович,NaT,аппарат искуственной вентиляции легких puritan...,004410124-0003335,2021-08-11,петайкиниванборисовичnat004410124-0003335,петайкин#NaT#004410124-0003335#2021-08-11,петайкин#NaT#004410124-0003335
1,петайкин иван борисович,NaT,концентратор кислородный jay-5a.,004410124-0003081,2021-08-11,петайкиниванборисовичnat004410124-0003081,петайкин#NaT#004410124-0003081#2021-08-11,петайкин#NaT#004410124-0003081
2,дыров александр иванович,2015-08-11,аппарат нивл wm 100 prisma30st,004410124-001815,2021-06-23,дыровалександриванович2015-08-11004410124-001815,дыров#2015-08-11#004410124-001815#2021-06-23,дыров#2015-08-11#004410124-001815
3,чиркова ангелина сергеевна,2021-01-02,аппарат искуственной вентиляции легких puritan...,004410124-0003266,2022-12-15,чирковаангелинасергеевна2021-01-02004410124-00...,чиркова#2021-01-02#004410124-0003266#2022-12-15,чиркова#2021-01-02#004410124-0003266
4,галан ринат андреевич,2007-04-20,аппарат нивл wm 100 prisma30st,004410124-001812,2022-05-26,галанринатандреевич2007-04-20004410124-001812,галан#2007-04-20#004410124-001812#2022-05-26,галан#2007-04-20#004410124-001812
...,...,...,...,...,...,...,...,...
821,боровков владимир иванович,1952-12-14,"концентратор кислорода invacare, вариант испол...",004410124-001252,2023-06-23,боровковвладимириванович1952-12-14004410124-00...,боровков#1952-12-14#004410124-001252#2023-06-23,боровков#1952-12-14#004410124-001252
822,баренбойм елена викторовна,1971-04-02,концентратор кислородный jay в следующем вариа...,004410134-0084495,2023-07-10,баренбоймеленавикторовна1971-04-02004410134-00...,баренбойм#1971-04-02#004410134-0084495#2023-07-10,баренбойм#1971-04-02#004410134-0084495
823,подгорный александр александрович,1962-11-24,концентратор кислородный jay-10,004410124-0004299,2023-06-26,подгорныйалександралександрович1962-11-2400441...,подгорный#1962-11-24#004410124-0004299#2023-06-26,подгорный#1962-11-24#004410124-0004299
824,обухова ольга ивановна,1946-11-03,концентратор кислородный jay в следующем вариа...,004410134-0084459,2023-06-28,обуховаольгаивановна1946-11-03004410134-0084459,обухова#1946-11-03#004410134-0084459#2023-06-28,обухова#1946-11-03#004410134-0084459


<div class="alert alert-block alert-info"><font size=3>
Удалить <b> ПОВТОРЫ ВОЗВРАТОВ пациент-дата возврата - прибор

In [872]:
data_vozvrat.shape

(826, 8)

In [873]:
data_vozvrat = data_vozvrat.drop_duplicates('Ключ_удлн_повт_возвр')
data_vozvrat.shape

(824, 8)

<div class="alert alert-block alert-danger"><font size=3>
<b>СОЕДИНИМ ВЫДАЧУ и ВОЗВРАТ           

In [874]:
# *****************************

In [875]:
df_poluchenie_nomenk

,Пациент_получ,ДР_получ,Пол_получ,Дата_выдачи_получ,Номенклатура_получ,Инвент_ном_получ,Ключ,Ключ_пац_прибор,Ключ_удлн_повт_выдачи
0,шваков макар олегович,2014-04-14,мужской,2021-04-14,концентратор кислородный jay-5a.,004410124-0003072,шваковмакаролегович2014-04-14004410124-0003072,шваков#2014-04-14#004410124-0003072,шваков#2014-04-14#004410124-0003072#2021-04-14
1,мартынов павел дмитриевич,2010-06-20,мужской,2021-03-17,аппарат для удаления выделений из дыхательных ...,004410124-0002667,мартыновпавелдмитриевич2010-06-20004410124-000...,мартынов#2010-06-20#004410124-0002667,мартынов#2010-06-20#004410124-0002667#2021-03-17
2,дарчиева кристина арсеновна,2010-03-15,женский,2021-03-26,аппарат искуственной вентиляции легких puritan...,004410124-0003302,дарчиевакристинаарсеновна2010-03-15004410124-0...,дарчиева#2010-03-15#004410124-0003302,дарчиева#2010-03-15#004410124-0003302#2021-03-26
3,бабаев асланхон акмалхонович,2019-06-05,мужской,2021-03-31,концентратор кислородный jay-5a.,004410124-0003073,бабаевасланхонакмалхонович2019-06-05004410124-...,бабаев#2019-06-05#004410124-0003073,бабаев#2019-06-05#004410124-0003073#2021-03-31
4,горячев владислав сергеевич,2006-12-28,мужской,2021-04-15,аппарат нивл wm 100 prisma30st,004410124-001791,горячеввладиславсергеевич2006-12-28004410124-0...,горячев#2006-12-28#004410124-001791,горячев#2006-12-28#004410124-001791#2021-04-15
...,...,...,...,...,...,...,...,...,...
21531,коваленко матвей сергеевич,2020-02-10,мужской,2023-07-14,аппарат искуственной вентиляции легких puritan...,004410124-0003331,коваленкоматвейсергеевич2020-02-10004410124-00...,коваленко#2020-02-10#004410124-0003331,коваленко#2020-02-10#004410124-0003331#2023-07-14
21533,медведев владимир зенонович,1953-01-10,мужской,2023-07-14,"концентратор кислорода invacare, вариант испол...",004410124-000513,медведеввладимирзенонович1953-01-10004410124-0...,медведев#1953-01-10#004410124-000513,медведев#1953-01-10#004410124-000513#2023-07-14
21538,расторгуева елена ивановна,1964-06-16,женский,2023-07-15,аппарат искусственной вентиляции легких для пр...,004210124-0000126,расторгуеваеленаивановна1964-06-16004210124-00...,расторгуева#1964-06-16#004210124-0000126,расторгуева#1964-06-16#004210124-0000126#2023-...
21558,агапов андрей васильевич,1962-03-10,мужской,2023-07-17,аппарат для лечения нарушений дыхания во сне w...,004410124-0006265,агаповандрейвасильевич1962-03-10004410124-0006265,агапов#1962-03-10#004410124-0006265,агапов#1962-03-10#004410124-0006265#2023-07-17


In [876]:
data_vozvrat

,Пациент_возвр,ДР_возвр,Номенклатура_возвр,Инвент_ном_возвр,Дата_возвр,Ключ,Ключ_удлн_повт_возвр,Ключ_пац_прибор
0,петайкин иван борисович,NaT,аппарат искуственной вентиляции легких puritan...,004410124-0003335,2021-08-11,петайкиниванборисовичnat004410124-0003335,петайкин#NaT#004410124-0003335#2021-08-11,петайкин#NaT#004410124-0003335
1,петайкин иван борисович,NaT,концентратор кислородный jay-5a.,004410124-0003081,2021-08-11,петайкиниванборисовичnat004410124-0003081,петайкин#NaT#004410124-0003081#2021-08-11,петайкин#NaT#004410124-0003081
2,дыров александр иванович,2015-08-11,аппарат нивл wm 100 prisma30st,004410124-001815,2021-06-23,дыровалександриванович2015-08-11004410124-001815,дыров#2015-08-11#004410124-001815#2021-06-23,дыров#2015-08-11#004410124-001815
3,чиркова ангелина сергеевна,2021-01-02,аппарат искуственной вентиляции легких puritan...,004410124-0003266,2022-12-15,чирковаангелинасергеевна2021-01-02004410124-00...,чиркова#2021-01-02#004410124-0003266#2022-12-15,чиркова#2021-01-02#004410124-0003266
4,галан ринат андреевич,2007-04-20,аппарат нивл wm 100 prisma30st,004410124-001812,2022-05-26,галанринатандреевич2007-04-20004410124-001812,галан#2007-04-20#004410124-001812#2022-05-26,галан#2007-04-20#004410124-001812
...,...,...,...,...,...,...,...,...
821,боровков владимир иванович,1952-12-14,"концентратор кислорода invacare, вариант испол...",004410124-001252,2023-06-23,боровковвладимириванович1952-12-14004410124-00...,боровков#1952-12-14#004410124-001252#2023-06-23,боровков#1952-12-14#004410124-001252
822,баренбойм елена викторовна,1971-04-02,концентратор кислородный jay в следующем вариа...,004410134-0084495,2023-07-10,баренбоймеленавикторовна1971-04-02004410134-00...,баренбойм#1971-04-02#004410134-0084495#2023-07-10,баренбойм#1971-04-02#004410134-0084495
823,подгорный александр александрович,1962-11-24,концентратор кислородный jay-10,004410124-0004299,2023-06-26,подгорныйалександралександрович1962-11-2400441...,подгорный#1962-11-24#004410124-0004299#2023-06-26,подгорный#1962-11-24#004410124-0004299
824,обухова ольга ивановна,1946-11-03,концентратор кислородный jay в следующем вариа...,004410134-0084459,2023-06-28,обуховаольгаивановна1946-11-03004410134-0084459,обухова#1946-11-03#004410134-0084459#2023-06-28,обухова#1946-11-03#004410134-0084459


In [877]:
# ************************
df_poluch_vozvtat = df_poluchenie_nomenk.merge(data_vozvrat, how = 'left', on='Ключ_пац_прибор')
# df_poluch_vozvtat = df_poluchenie_nomenk.merge(data_vozvrat, how = 'left')
df_poluch_vozvtat

,Пациент_получ,ДР_получ,Пол_получ,Дата_выдачи_получ,Номенклатура_получ,Инвент_ном_получ,Ключ_x,Ключ_пац_прибор,Ключ_удлн_повт_выдачи,Пациент_возвр,ДР_возвр,Номенклатура_возвр,Инвент_ном_возвр,Дата_возвр,Ключ_y,Ключ_удлн_повт_возвр
0,шваков макар олегович,2014-04-14,мужской,2021-04-14,концентратор кислородный jay-5a.,004410124-0003072,шваковмакаролегович2014-04-14004410124-0003072,шваков#2014-04-14#004410124-0003072,шваков#2014-04-14#004410124-0003072#2021-04-14,NaN,NaT,NaN,NaN,NaT,NaN,NaN
1,мартынов павел дмитриевич,2010-06-20,мужской,2021-03-17,аппарат для удаления выделений из дыхательных ...,004410124-0002667,мартыновпавелдмитриевич2010-06-20004410124-000...,мартынов#2010-06-20#004410124-0002667,мартынов#2010-06-20#004410124-0002667#2021-03-17,NaN,NaT,NaN,NaN,NaT,NaN,NaN
2,дарчиева кристина арсеновна,2010-03-15,женский,2021-03-26,аппарат искуственной вентиляции легких puritan...,004410124-0003302,дарчиевакристинаарсеновна2010-03-15004410124-0...,дарчиева#2010-03-15#004410124-0003302,дарчиева#2010-03-15#004410124-0003302#2021-03-26,NaN,NaT,NaN,NaN,NaT,NaN,NaN
3,бабаев асланхон акмалхонович,2019-06-05,мужской,2021-03-31,концентратор кислородный jay-5a.,004410124-0003073,бабаевасланхонакмалхонович2019-06-05004410124-...,бабаев#2019-06-05#004410124-0003073,бабаев#2019-06-05#004410124-0003073#2021-03-31,бабаев асланхон акмалхонович,2019-06-05,концентратор кислородный jay-5a.,004410124-0003073,2021-07-19,бабаевасланхонакмалхонович2019-06-05004410124-...,бабаев#2019-06-05#004410124-0003073#2021-07-19
4,горячев владислав сергеевич,2006-12-28,мужской,2021-04-15,аппарат нивл wm 100 prisma30st,004410124-001791,горячеввладиславсергеевич2006-12-28004410124-0...,горячев#2006-12-28#004410124-001791,горячев#2006-12-28#004410124-001791#2021-04-15,NaN,NaT,NaN,NaN,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3038,коваленко матвей сергеевич,2020-02-10,мужской,2023-07-14,аппарат искуственной вентиляции легких puritan...,004410124-0003331,коваленкоматвейсергеевич2020-02-10004410124-00...,коваленко#2020-02-10#004410124-0003331,коваленко#2020-02-10#004410124-0003331#2023-07-14,NaN,NaT,NaN,NaN,NaT,NaN,NaN
3039,медведев владимир зенонович,1953-01-10,мужской,2023-07-14,"концентратор кислорода invacare, вариант испол...",004410124-000513,медведеввладимирзенонович1953-01-10004410124-0...,медведев#1953-01-10#004410124-000513,медведев#1953-01-10#004410124-000513#2023-07-14,NaN,NaT,NaN,NaN,NaT,NaN,NaN
3040,расторгуева елена ивановна,1964-06-16,женский,2023-07-15,аппарат искусственной вентиляции легких для пр...,004210124-0000126,расторгуеваеленаивановна1964-06-16004210124-00...,расторгуева#1964-06-16#004210124-0000126,расторгуева#1964-06-16#004210124-0000126#2023-...,NaN,NaT,NaN,NaN,NaT,NaN,NaN
3041,агапов андрей васильевич,1962-03-10,мужской,2023-07-17,аппарат для лечения нарушений дыхания во сне w...,004410124-0006265,агаповандрейвасильевич1962-03-10004410124-0006265,агапов#1962-03-10#004410124-0006265,агапов#1962-03-10#004410124-0006265#2023-07-17,NaN,NaT,NaN,NaN,NaT,NaN,NaN


In [878]:
# -------------- проверяем, что есть каки-то выдачи с возвратами РАНЕЕ получение ??? Удалим такие кривизын
df_poluch_vozvtat.loc[df_poluch_vozvtat['Дата_возвр'] < df_poluch_vozvtat['Дата_выдачи_получ']]

,Пациент_получ,ДР_получ,Пол_получ,Дата_выдачи_получ,Номенклатура_получ,Инвент_ном_получ,Ключ_x,Ключ_пац_прибор,Ключ_удлн_повт_выдачи,Пациент_возвр,ДР_возвр,Номенклатура_возвр,Инвент_ном_возвр,Дата_возвр,Ключ_y,Ключ_удлн_повт_возвр
570,квиркелия татьяна васильевна,1948-04-10,женский,2022-05-29,матрас противопролежневый (ячеистый).,00,квиркелиятатьянавасильевна1948-04-1000,квиркелия#1948-04-10#00#матрас противопролежне...,квиркелия#1948-04-10#00#матрас противопролежне...,квиркелия татьяна васильевна,1948-04-10,матрас противопролежневый (ячеистый).,00,2022-05-12,квиркелиятатьянавасильевна1948-04-1000,квиркелия#1948-04-10#00#матрас противопролежне...
1867,сотникова полина алексеевна,2002-12-03,женский,2023-01-25,аппарат искусственной вентиляции легких для пр...,004210124-0000078,сотниковаполинаалексеевна2002-12-03004210124-0...,сотникова#2002-12-03#004210124-0000078,сотникова#2002-12-03#004210124-0000078#2023-01-25,сотникова полина алексеевна,2002-12-03,аппарат искусственной вентиляции легких для пр...,004210124-0000078,2023-01-19,сотниковаполинаалексеевна2002-12-03004210124-0...,сотникова#2002-12-03#004210124-0000078#2023-01-19
1955,грошева оксана владимировна,1987-08-06,женский,2023-03-01,"концентратор кислорода invacare, вариант испол...",004410124-000943,грошеваоксанавладимировна1987-08-06004410124-0...,грошева#1987-08-06#004410124-000943,грошева#1987-08-06#004410124-000943#2023-03-01,грошева оксана владимировна,1987-08-06,"концентратор кислорода invacare, вариант испол...",004410124-000943,2023-02-20,грошеваоксанавладимировна1987-08-06004410124-0...,грошева#1987-08-06#004410124-000943#2023-02-20
2037,дубинская наталья петровна,1949-04-25,женский,2023-02-28,отсасыватель хирургический аrmed 7е-а,б/н,дубинскаянатальяпетровна1949-04-25б/н,дубинская#1949-04-25#б/н#отсасыватель хирургич...,дубинская#1949-04-25#б/н#отсасыватель хирургич...,дубинская наталья петровна,1949-04-25,отсасыватель хирургический аrmed 7е-а,б/н,2023-02-27,дубинскаянатальяпетровна1949-04-25б/н,дубинская#1949-04-25#б/н#отсасыватель хирургич...
2439,мальцев анатолий фролович,1950-01-02,мужской,2023-04-03,"концентратор кислорода invacare, вариант испол...",004410124-000893,мальцеванатолийфролович1950-01-02004410124-000893,мальцев#1950-01-02#004410124-000893,мальцев#1950-01-02#004410124-000893#2023-04-03,мальцев анатолий фролович,1950-01-02,"концентратор кислорода invacare, вариант испол...",004410124-000893,2023-03-30,мальцеванатолийфролович1950-01-02004410124-000893,мальцев#1950-01-02#004410124-000893#2023-03-30
2600,ефремов сергей ильич,1964-08-18,мужской,2023-07-04,концентратор кислородный jay-5a.,004410134-0081647,ефремовсергейильич1964-08-18004410134-0081647,ефремов#1964-08-18#004410134-0081647,ефремов#1964-08-18#004410134-0081647#2023-07-04,ефремов сергей ильич,1964-08-18,концентратор кислородный jay-5a.,004410134-0081647,2023-07-03,ефремовсергейильич1964-08-18004410134-0081647,ефремов#1964-08-18#004410134-0081647#2023-07-03
2628,рубанов станислав степанович,1940-08-17,мужской,2023-05-26,отсасыватель хирургический аrmed 7е-а,б/н,рубановстаниславстепанович1940-08-17б/н,рубанов#1940-08-17#б/н#отсасыватель хирургичес...,рубанов#1940-08-17#б/н#отсасыватель хирургичес...,рубанов станислав степанович,1940-08-17,отсасыватель хирургический аrmed 7е-а,б/н,2023-05-25,рубановстаниславстепанович1940-08-17б/н,рубанов#1940-08-17#б/н#отсасыватель хирургичес...
2629,рубанов станислав степанович,1940-08-17,мужской,2023-05-26,отсасыватель хирургический аrmed 7е-а,б/н,рубановстаниславстепанович1940-08-17б/н,рубанов#1940-08-17#б/н#отсасыватель хирургичес...,рубанов#1940-08-17#б/н#отсасыватель хирургичес...,рубанов станислав степанович,1940-08-17,отсасыватель хирургический аrmed 7е-а,б/н,2022-11-03,рубановстаниславстепанович1940-08-17б/н,рубанов#1940-08-17#б/н#отсасыватель хирургичес...
2724,алборова ирина вениаминовна,1962-11-25,женский,2023-07-10,концентратор кислородный jay в следующем вариа...,004410124-0003179,албороваиринавениаминовна1962-11-25004410124-0...,алборова#1962

<div class="alert alert-block alert-info"><font size=3>
Если дата ВОЗВРАТА <b>ПОЗЖЕ</b> чем ПОЛУЧЕНИЕ, то Удалим ДАТУ ВОЗВРАТА- это 100% ошибка 

In [879]:
df_poluch_vozvtat.loc[df_poluch_vozvtat['Дата_возвр'] < df_poluch_vozvtat['Дата_выдачи_получ'], 'Дата_возвр'] = np.nan
df_poluch_vozvtat

,Пациент_получ,ДР_получ,Пол_получ,Дата_выдачи_получ,Номенклатура_получ,Инвент_ном_получ,Ключ_x,Ключ_пац_прибор,Ключ_удлн_повт_выдачи,Пациент_возвр,ДР_возвр,Номенклатура_возвр,Инвент_ном_возвр,Дата_возвр,Ключ_y,Ключ_удлн_повт_возвр
0,шваков макар олегович,2014-04-14,мужской,2021-04-14,концентратор кислородный jay-5a.,004410124-0003072,шваковмакаролегович2014-04-14004410124-0003072,шваков#2014-04-14#004410124-0003072,шваков#2014-04-14#004410124-0003072#2021-04-14,NaN,NaT,NaN,NaN,NaT,NaN,NaN
1,мартынов павел дмитриевич,2010-06-20,мужской,2021-03-17,аппарат для удаления выделений из дыхательных ...,004410124-0002667,мартыновпавелдмитриевич2010-06-20004410124-000...,мартынов#2010-06-20#004410124-0002667,мартынов#2010-06-20#004410124-0002667#2021-03-17,NaN,NaT,NaN,NaN,NaT,NaN,NaN
2,дарчиева кристина арсеновна,2010-03-15,женский,2021-03-26,аппарат искуственной вентиляции легких puritan...,004410124-0003302,дарчиевакристинаарсеновна2010-03-15004410124-0...,дарчиева#2010-03-15#004410124-0003302,дарчиева#2010-03-15#004410124-0003302#2021-03-26,NaN,NaT,NaN,NaN,NaT,NaN,NaN
3,бабаев асланхон акмалхонович,2019-06-05,мужской,2021-03-31,концентратор кислородный jay-5a.,004410124-0003073,бабаевасланхонакмалхонович2019-06-05004410124-...,бабаев#2019-06-05#004410124-0003073,бабаев#2019-06-05#004410124-0003073#2021-03-31,бабаев асланхон акмалхонович,2019-06-05,концентратор кислородный jay-5a.,004410124-0003073,2021-07-19,бабаевасланхонакмалхонович2019-06-05004410124-...,бабаев#2019-06-05#004410124-0003073#2021-07-19
4,горячев владислав сергеевич,2006-12-28,мужской,2021-04-15,аппарат нивл wm 100 prisma30st,004410124-001791,горячеввладиславсергеевич2006-12-28004410124-0...,горячев#2006-12-28#004410124-001791,горячев#2006-12-28#004410124-001791#2021-04-15,NaN,NaT,NaN,NaN,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3038,коваленко матвей сергеевич,2020-02-10,мужской,2023-07-14,аппарат искуственной вентиляции легких puritan...,004410124-0003331,коваленкоматвейсергеевич2020-02-10004410124-00...,коваленко#2020-02-10#004410124-0003331,коваленко#2020-02-10#004410124-0003331#2023-07-14,NaN,NaT,NaN,NaN,NaT,NaN,NaN
3039,медведев владимир зенонович,1953-01-10,мужской,2023-07-14,"концентратор кислорода invacare, вариант испол...",004410124-000513,медведеввладимирзенонович1953-01-10004410124-0...,медведев#1953-01-10#004410124-000513,медведев#1953-01-10#004410124-000513#2023-07-14,NaN,NaT,NaN,NaN,NaT,NaN,NaN
3040,расторгуева елена ивановна,1964-06-16,женский,2023-07-15,аппарат искусственной вентиляции легких для пр...,004210124-0000126,расторгуеваеленаивановна1964-06-16004210124-00...,расторгуева#1964-06-16#004210124-0000126,расторгуева#1964-06-16#004210124-0000126#2023-...,NaN,NaT,NaN,NaN,NaT,NaN,NaN
3041,агапов андрей васильевич,1962-03-10,мужской,2023-07-17,аппарат для лечения нарушений дыхания во сне w...,004410124-0006265,агаповандрейвасильевич1962-03-10004410124-0006265,агапов#1962-03-10#004410124-0006265,агапов#1962-03-10#004410124-0006265#2023-07-17,NaN,NaT,NaN,NaN,NaT,NaN,NaN


In [880]:
# df_poluch_vozvtat.loc[df_poluch_vozvtat['Пациент_получ'] == 'сотникова полина алексеевна']
# df_poluch_vozvtat.loc[df_poluch_vozvtat['Пациент_получ'] == 'дубинская наталья петровна']

<div class="alert alert-block alert-info"><font size=3>
ПЕРЕИМЕНУЕМ           </div>

In [881]:
df_poluch_vozvtat.columns

Index(['Пациент_получ', 'ДР_получ', 'Пол_получ', 'Дата_выдачи_получ',
       'Номенклатура_получ', 'Инвент_ном_получ', 'Ключ_x', 'Ключ_пац_прибор',
       'Ключ_удлн_повт_выдачи', 'Пациент_возвр', 'ДР_возвр',
       'Номенклатура_возвр', 'Инвент_ном_возвр', 'Дата_возвр', 'Ключ_y',
       'Ключ_удлн_повт_возвр'],
      dtype='object')

In [882]:
df_poluch_vozvtat = df_poluch_vozvtat[[
    'Пациент_получ',
    'ДР_получ',
        'Дата_выдачи_получ',
        'Дата_возвр',
    # 'Пол_получ', 

       'Номенклатура_получ', 
    'Инвент_ном_получ',
    
    'Ключ_пац_прибор',
       # 'Ключ_удлн_повт_выдачи',
    # 'Пациент_возвр',
    # 'ДР_возвр',
       # 'Номенклатура_возвр',
    # 'Инвент_ном_возвр',

    # 'Ключ_y',
       # 'Ключ_удлн_повт_возвр'
]]
df_poluch_vozvtat.rename(columns={'Пациент_получ': 'Пациент'}, inplace=True)
df_poluch_vozvtat.rename(columns={'ДР_получ': 'ДР'}, inplace=True)
df_poluch_vozvtat.rename(columns={'Номенклатура_получ': 'Номенклатура'}, inplace=True)
df_poluch_vozvtat.rename(columns={'Инвент_ном_получ': 'Инвент_ном'}, inplace=True)
df_poluch_vozvtat.rename(columns={'Дата_выдачи_получ': 'Дата_выдачи'}, inplace=True)
df_poluch_vozvtat

,Пациент,ДР,Дата_выдачи,Дата_возвр,Номенклатура,Инвент_ном,Ключ_пац_прибор
0,шваков макар олегович,2014-04-14,2021-04-14,NaT,концентратор кислородный jay-5a.,004410124-0003072,шваков#2014-04-14#004410124-0003072
1,мартынов павел дмитриевич,2010-06-20,2021-03-17,NaT,аппарат для удаления выделений из дыхательных ...,004410124-0002667,мартынов#2010-06-20#004410124-0002667
2,дарчиева кристина арсеновна,2010-03-15,2021-03-26,NaT,аппарат искуственной вентиляции легких puritan...,004410124-0003302,дарчиева#2010-03-15#004410124-0003302
3,бабаев асланхон акмалхонович,2019-06-05,2021-03-31,2021-07-19,концентратор кислородный jay-5a.,004410124-0003073,бабаев#2019-06-05#004410124-0003073
4,горячев владислав сергеевич,2006-12-28,2021-04-15,NaT,аппарат нивл wm 100 prisma30st,004410124-001791,горячев#2006-12-28#004410124-001791
...,...,...,...,...,...,...,...
3038,коваленко матвей сергеевич,2020-02-10,2023-07-14,NaT,аппарат искуственной вентиляции легких puritan...,004410124-0003331,коваленко#2020-02-10#004410124-0003331
3039,медведев владимир зенонович,1953-01-10,2023-07-14,NaT,"концентратор кислорода invacare, вариант испол...",004410124-000513,медведев#1953-01-10#004410124-000513
3040,расторгуева елена ивановна,1964-06-16,2023-07-15,NaT,аппарат искусственной вентиляции легких для пр...,004210124-0000126,расторгуева#1964-06-16#004210124-0000126
3041,агапов андрей васильевич,1962-03-10,2023-07-17,NaT,аппарат для лечения нарушений дыхания во сне w...,004410124-0006265,агапов#1962-03-10#004410124-0006265


In [883]:
df_poluch_vozvtat.shape

(3043, 7)

In [884]:
# --------------- ПОЛУЧЕНИЕ ДАТАФРЕЙМА  БЕЗ НУЛЕВЫХ ВЫДАЧ НОМЕНКЛАТУР
df_poluch_vozvtat_not_nul_nomenklat = df_poluch_vozvtat.loc[~df_poluch_vozvtat['Номенклатура'].isna()]
df_poluch_vozvtat_not_nul_nomenklat

,Пациент,ДР,Дата_выдачи,Дата_возвр,Номенклатура,Инвент_ном,Ключ_пац_прибор
0,шваков макар олегович,2014-04-14,2021-04-14,NaT,концентратор кислородный jay-5a.,004410124-0003072,шваков#2014-04-14#004410124-0003072
1,мартынов павел дмитриевич,2010-06-20,2021-03-17,NaT,аппарат для удаления выделений из дыхательных ...,004410124-0002667,мартынов#2010-06-20#004410124-0002667
2,дарчиева кристина арсеновна,2010-03-15,2021-03-26,NaT,аппарат искуственной вентиляции легких puritan...,004410124-0003302,дарчиева#2010-03-15#004410124-0003302
3,бабаев асланхон акмалхонович,2019-06-05,2021-03-31,2021-07-19,концентратор кислородный jay-5a.,004410124-0003073,бабаев#2019-06-05#004410124-0003073
4,горячев владислав сергеевич,2006-12-28,2021-04-15,NaT,аппарат нивл wm 100 prisma30st,004410124-001791,горячев#2006-12-28#004410124-001791
...,...,...,...,...,...,...,...
3038,коваленко матвей сергеевич,2020-02-10,2023-07-14,NaT,аппарат искуственной вентиляции легких puritan...,004410124-0003331,коваленко#2020-02-10#004410124-0003331
3039,медведев владимир зенонович,1953-01-10,2023-07-14,NaT,"концентратор кислорода invacare, вариант испол...",004410124-000513,медведев#1953-01-10#004410124-000513
3040,расторгуева елена ивановна,1964-06-16,2023-07-15,NaT,аппарат искусственной вентиляции легких для пр...,004210124-0000126,расторгуева#1964-06-16#004210124-0000126
3041,агапов андрей васильевич,1962-03-10,2023-07-17,NaT,аппарат для лечения нарушений дыхания во сне w...,004410124-0006265,агапов#1962-03-10#004410124-0006265


####
<div class="alert alert-block alert-success">
    <!-- -info-warning-success-danger>   -->
<font size=5>
<b>Данные по выдачам - возвратам из 1C
    </b>    </div>
<hr>

In [885]:
df_1c = df_poluch_vozvtat_not_nul_nomenklat.copy()

In [886]:
df_1c.to_excel('df_1c.xlsx')

####
<div class="alert alert-block alert-info">
    <!-- -info-warning-success-danger>   -->
<font size=5>
<b>Подключение гугл-таблиц сбора даных о выдаче
    </b>    </div>
<hr>

In [887]:
data_df = pd.read_csv('Респираторка_гугл_2.csv', delimiter=';', encoding='cp1251')
data_df = def_low(data_df)
print(data_df.shape)

(12041, 10)


####
<div class="alert alert-block alert-warning">
    <!-- -info-warning-success-danger>   -->
<font size=5>
<b>Первичное поступление на склад ВЫКИДЫВАЕМ!!!!
    </b>    </div>
<hr>

In [888]:
df_google = data_df.loc[data_df['Тип движения ТСР'] != 'первичное поступление на склад']
df_google.shape

(10040, 10)

In [889]:
#  нормализация дат
df_google['Дата рождения'] = pd.to_datetime(df_google['Дата рождения'], dayfirst=True, errors = 'coerce').dt.normalize()
df_google['Дата выдачи'] = pd.to_datetime(df_google['Дата выдачи'], dayfirst=True, errors = 'coerce').dt.normalize()
df_google['Дата возврата'] = pd.to_datetime(df_google['Дата возврата'], dayfirst=True, errors = 'coerce').dt.normalize()
df_google.head(3)

,Тип оборудования,Единое наименование,Инвентарный номер,Тип движения ТСР,Фамилия,Имя,Отчество,Дата рождения,Дата выдачи,Дата возврата
1,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,выдано пациенту,степлякова,мария,григорьевна,1940-07-08,2021-04-24,NaT
2,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,возврат на склад,степлякова,мария,григорьевна,1940-07-08,2021-04-24,2021-05-10
3,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,выдано пациенту,селиверстов,аркадий,иванович,1946-03-25,2021-05-11,NaT


<div class="alert alert-block alert-info"><font size=3>
Ключ с инвентарым и датой выдачи           </div>

In [890]:
# df_google.loc[df_google['Инвентарный номер'].str.contains('-') == False]['Инвентарный номер'].unique()

In [891]:
# df_google.loc[df_google['Инвентарный номер'].str.contains('0514') == True].head(50)

In [892]:
   #  ----- Добавим УНИКАЛЬНЫЙ ключ ФИО+ДР - без пробелов ------------
df_google['Ключ_пац_прибор'] = df_google['Фамилия'] + '#' + (df_google['Дата рождения'].dt.date).map(str) + '#' + df_google['Инвентарный номер'] + '#' + (df_google['Дата выдачи'].dt.date).map(str) 
df_google['Ключ_пац_прибор'] = df_google['Ключ_пац_прибор'].str.lower()            # ---- все к нижнему регистру
df_google['Ключ_пац_прибор'] = df_google['Ключ_пац_прибор'].str.replace(' ', '')   # ---- пробелы долой 
df_google.head(3)

,Тип оборудования,Единое наименование,Инвентарный номер,Тип движения ТСР,Фамилия,Имя,Отчество,Дата рождения,Дата выдачи,Дата возврата,Ключ_пац_прибор
1,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,выдано пациенту,степлякова,мария,григорьевна,1940-07-08,2021-04-24,NaT,степлякова#1940-07-08#004410124-0002849#2021-0...
2,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,возврат на склад,степлякова,мария,григорьевна,1940-07-08,2021-04-24,2021-05-10,степлякова#1940-07-08#004410124-0002849#2021-0...
3,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,выдано пациенту,селиверстов,аркадий,иванович,1946-03-25,2021-05-11,NaT,селиверстов#1946-03-25#004410124-0002849#2021-...


<div class="alert alert-block alert-info"><font size=3>
Ключ   ПАЦИЕНТ - ПРИБОР         </div>

In [893]:
# ------------------ Новый ключ ПАЦИЕНТ - ПРИБОР --------------
# df_google.reset_index(drop= True , inplace= True )
df_google.loc[df_google['Инвентарный номер'].str.contains('-') == False, 'Ключ_пац_прибор' ] = df_google['Фамилия'] + '#' + (df_google['Дата рождения'].dt.date).map(str) +  '#' + df_google['Инвентарный номер'] +  '#' + df_google['Единое наименование'] 

df_google.loc[df_google['Инвентарный номер'].str.contains('-') == True, 'Ключ_пац_прибор' ] = df_google['Фамилия'] + '#' + (df_google['Дата рождения'].dt.date).map(str) +  '#' + df_google['Инвентарный номер'] 

<div class="alert alert-block alert-warning"><font size=3>
Датафрейм с ТОЛЬКО ПОЛУЧЕНИЕ     </div>

In [894]:
df_google_only_poluch = df_google.loc[df_google['Дата возврата'].isna()]
df_google_only_poluch

,Тип оборудования,Единое наименование,Инвентарный номер,Тип движения ТСР,Фамилия,Имя,Отчество,Дата рождения,Дата выдачи,Дата возврата,Ключ_пац_прибор
1,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,выдано пациенту,степлякова,мария,григорьевна,1940-07-08,2021-04-24,NaT,степлякова#1940-07-08#004410124-0002849
3,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,выдано пациенту,селиверстов,аркадий,иванович,1946-03-25,2021-05-11,NaT,селиверстов#1946-03-25#004410124-0002849
5,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,выдано пациенту,никитина,татьяна,николаевна,1940-06-11,2021-09-24,NaT,никитина#1940-06-11#004410124-0002849
7,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,выдано пациенту,спирина,ираида,васильевна,1938-09-24,2021-10-21,NaT,спирина#1938-09-24#004410124-0002849
9,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,выдано пациенту,бадамшин,сергей,чингисович,1954-08-13,2021-10-27,NaT,бадамшин#1954-08-13#004410124-0002849
...,...,...,...,...,...,...,...,...,...,...,...
11553,концентратор,"концентратор кислорода invacare, вариант испол...",004410124-000513,выдано пациенту,медведев,владимир,зенонович,1953-01-10,2023-07-14,NaT,медведев#1953-01-10#004410124-000513
11554,нивл,аппарат искусственной вентиляции легких для пр...,004210124-0000126,выдано пациенту,расторгуева,елена,ивановна,1946-06-16,2023-07-15,NaT,расторгуева#1946-06-16#004210124-0000126
11555,аспиратор,"отсасыватель хирургический электрический ""arme...",б/н,выдано пациенту,матюшин,сергей,олегович,2002-11-14,2023-07-17,NaT,матюшин#2002-11-14#б/н#отсасыватель хирургичес...
11556,пульсоксиметр,пульсоксиметр (аппарат мониторинга пациента) n...,004410124-0006370,выдано пациенту,матюшин,сергей,олегович,2002-11-14,2023-07-17,NaT,матюшин#2002-11-14#004410124-0006370


In [895]:
# -------------------------удалим повторные записи по ключу 
df_google_only_poluch = df_google_only_poluch.drop_duplicates('Ключ_пац_прибор')
df_google_only_poluch

,Тип оборудования,Единое наименование,Инвентарный номер,Тип движения ТСР,Фамилия,Имя,Отчество,Дата рождения,Дата выдачи,Дата возврата,Ключ_пац_прибор
1,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,выдано пациенту,степлякова,мария,григорьевна,1940-07-08,2021-04-24,NaT,степлякова#1940-07-08#004410124-0002849
3,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,выдано пациенту,селиверстов,аркадий,иванович,1946-03-25,2021-05-11,NaT,селиверстов#1946-03-25#004410124-0002849
5,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,выдано пациенту,никитина,татьяна,николаевна,1940-06-11,2021-09-24,NaT,никитина#1940-06-11#004410124-0002849
7,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,выдано пациенту,спирина,ираида,васильевна,1938-09-24,2021-10-21,NaT,спирина#1938-09-24#004410124-0002849
9,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,выдано пациенту,бадамшин,сергей,чингисович,1954-08-13,2021-10-27,NaT,бадамшин#1954-08-13#004410124-0002849
...,...,...,...,...,...,...,...,...,...,...,...
11553,концентратор,"концентратор кислорода invacare, вариант испол...",004410124-000513,выдано пациенту,медведев,владимир,зенонович,1953-01-10,2023-07-14,NaT,медведев#1953-01-10#004410124-000513
11554,нивл,аппарат искусственной вентиляции легких для пр...,004210124-0000126,выдано пациенту,расторгуева,елена,ивановна,1946-06-16,2023-07-15,NaT,расторгуева#1946-06-16#004210124-0000126
11555,аспиратор,"отсасыватель хирургический электрический ""arme...",б/н,выдано пациенту,матюшин,сергей,олегович,2002-11-14,2023-07-17,NaT,матюшин#2002-11-14#б/н#отсасыватель хирургичес...
11556,пульсоксиметр,пульсоксиметр (аппарат мониторинга пациента) n...,004410124-0006370,выдано пациенту,матюшин,сергей,олегович,2002-11-14,2023-07-17,NaT,матюшин#2002-11-14#004410124-0006370


<div class="alert alert-block alert-warning"><font size=3>
Датафрейм с непустой датой ВОЗВРАТА          </div>

In [896]:
df_google_vozvrat = df_google.loc[~df_google['Дата возврата'].isna()]
df_google_vozvrat

,Тип оборудования,Единое наименование,Инвентарный номер,Тип движения ТСР,Фамилия,Имя,Отчество,Дата рождения,Дата выдачи,Дата возврата,Ключ_пац_прибор
2,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,возврат на склад,степлякова,мария,григорьевна,1940-07-08,2021-04-24,2021-05-10,степлякова#1940-07-08#004410124-0002849
4,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,возврат на склад,селиверстов,аркадий,иванович,1946-03-25,2021-05-11,2021-09-19,селиверстов#1946-03-25#004410124-0002849
6,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,возврат на склад,никитина,татьяна,николаевна,1940-06-11,2021-09-24,2021-10-20,никитина#1940-06-11#004410124-0002849
8,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,возврат на склад,спирина,ираида,васильевна,1938-09-24,2021-10-21,2021-10-26,спирина#1938-09-24#004410124-0002849
12,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002850,возврат на склад,миронова,валентина,ивановна,1948-01-16,2021-11-15,2022-04-24,миронова#1948-01-16#004410124-0002850
...,...,...,...,...,...,...,...,...,...,...,...
11501,концентратор,концентратор кислородный jay - 10,004410124-0003234,возврат на склад,жук,людмила,александровна,1968-09-18,2023-04-14,2023-07-10,жук#1968-09-18#004410124-0003234
11530,концентратор,концентратор кислородный jay - 10,004410124-0003223,возврат на склад,нигруца,хамдия,NaN,1952-06-07,2023-06-05,2023-07-06,нигруца#1952-06-07#004410124-0003223
11538,концентратор,концентратор кислородный jay в следующем вариа...,004410134-0084489,возврат на склад,аксенов,борис,николаевич,1955-04-09,2023-05-04,2023-06-26,аксенов#1955-04-09#004410134-0084489
11542,концентратор,концентратор кислородный jay в следующем вариа...,004410134-0084495,возврат на склад,баренбойм,елена,викторовна,1971-04-02,2023-06-16,2023-07-11,баренбойм#1971-04-02#004410134-0084495


In [897]:
# df_google_vozvrat[df_google_vozvrat.duplicated('Ключ', keep=False)].sort_values('Ключ')

<div class="alert alert-block alert-info"><font size=3>
Уникальные записи с датой ВОЗВРАТА          </div>

In [898]:
# ----------------- удалим ПОВТОРЫ по строке КЛЮЧ с ВЫДАЧЕЙ НОМЕРА
df_google_vozvrat = df_google_vozvrat.drop_duplicates('Ключ_пац_прибор')
df_google_vozvrat

,Тип оборудования,Единое наименование,Инвентарный номер,Тип движения ТСР,Фамилия,Имя,Отчество,Дата рождения,Дата выдачи,Дата возврата,Ключ_пац_прибор
2,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,возврат на склад,степлякова,мария,григорьевна,1940-07-08,2021-04-24,2021-05-10,степлякова#1940-07-08#004410124-0002849
4,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,возврат на склад,селиверстов,аркадий,иванович,1946-03-25,2021-05-11,2021-09-19,селиверстов#1946-03-25#004410124-0002849
6,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,возврат на склад,никитина,татьяна,николаевна,1940-06-11,2021-09-24,2021-10-20,никитина#1940-06-11#004410124-0002849
8,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,возврат на склад,спирина,ираида,васильевна,1938-09-24,2021-10-21,2021-10-26,спирина#1938-09-24#004410124-0002849
12,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002850,возврат на склад,миронова,валентина,ивановна,1948-01-16,2021-11-15,2022-04-24,миронова#1948-01-16#004410124-0002850
...,...,...,...,...,...,...,...,...,...,...,...
11501,концентратор,концентратор кислородный jay - 10,004410124-0003234,возврат на склад,жук,людмила,александровна,1968-09-18,2023-04-14,2023-07-10,жук#1968-09-18#004410124-0003234
11530,концентратор,концентратор кислородный jay - 10,004410124-0003223,возврат на склад,нигруца,хамдия,NaN,1952-06-07,2023-06-05,2023-07-06,нигруца#1952-06-07#004410124-0003223
11538,концентратор,концентратор кислородный jay в следующем вариа...,004410134-0084489,возврат на склад,аксенов,борис,николаевич,1955-04-09,2023-05-04,2023-06-26,аксенов#1955-04-09#004410134-0084489
11542,концентратор,концентратор кислородный jay в следующем вариа...,004410134-0084495,возврат на склад,баренбойм,елена,викторовна,1971-04-02,2023-06-16,2023-07-11,баренбойм#1971-04-02#004410134-0084495


<div class="alert alert-block alert-warning"><font size=3>
создам df из ВОЗВРАТА только с данными   </div>

In [899]:
df_google_vozvrat.columns

Index(['Тип оборудования', 'Единое наименование', 'Инвентарный номер',
       'Тип движения ТСР', 'Фамилия', 'Имя', 'Отчество', 'Дата рождения',
       'Дата выдачи', 'Дата возврата', 'Ключ_пац_прибор'],
      dtype='object')

In [900]:
df_google_vozvrat_sh = df_google_vozvrat[[     'Дата выдачи', 'Дата возврата', 'Ключ_пац_прибор']] 
df_google_vozvrat_sh

,Дата выдачи,Дата возврата,Ключ_пац_прибор
2,2021-04-24,2021-05-10,степлякова#1940-07-08#004410124-0002849
4,2021-05-11,2021-09-19,селиверстов#1946-03-25#004410124-0002849
6,2021-09-24,2021-10-20,никитина#1940-06-11#004410124-0002849
8,2021-10-21,2021-10-26,спирина#1938-09-24#004410124-0002849
12,2021-11-15,2022-04-24,миронова#1948-01-16#004410124-0002850
...,...,...,...
11501,2023-04-14,2023-07-10,жук#1968-09-18#004410124-0003234
11530,2023-06-05,2023-07-06,нигруца#1952-06-07#004410124-0003223
11538,2023-05-04,2023-06-26,аксенов#1955-04-09#004410134-0084489
11542,2023-06-16,2023-07-11,баренбойм#1971-04-02#004410134-0084495


<div class="alert alert-block alert-info"><font size=3>
Соединим данные по выдаче-возврату 

In [901]:
df_google = df_google_only_poluch.merge(df_google_vozvrat_sh, how='left', on='Ключ_пац_прибор')
df_google

,Тип оборудования,Единое наименование,Инвентарный номер,Тип движения ТСР,Фамилия,Имя,Отчество,Дата рождения,Дата выдачи_x,Дата возврата_x,Ключ_пац_прибор,Дата выдачи_y,Дата возврата_y
0,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,выдано пациенту,степлякова,мария,григорьевна,1940-07-08,2021-04-24,NaT,степлякова#1940-07-08#004410124-0002849,2021-04-24,2021-05-10
1,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,выдано пациенту,селиверстов,аркадий,иванович,1946-03-25,2021-05-11,NaT,селиверстов#1946-03-25#004410124-0002849,2021-05-11,2021-09-19
2,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,выдано пациенту,никитина,татьяна,николаевна,1940-06-11,2021-09-24,NaT,никитина#1940-06-11#004410124-0002849,2021-09-24,2021-10-20
3,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,выдано пациенту,спирина,ираида,васильевна,1938-09-24,2021-10-21,NaT,спирина#1938-09-24#004410124-0002849,2021-10-21,2021-10-26
4,концентратор,концентратор кислородный jay в следующем вариа...,004410124-0002849,выдано пациенту,бадамшин,сергей,чингисович,1954-08-13,2021-10-27,NaT,бадамшин#1954-08-13#004410124-0002849,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6252,концентратор,"концентратор кислорода invacare, вариант испол...",004410124-000513,выдано пациенту,медведев,владимир,зенонович,1953-01-10,2023-07-14,NaT,медведев#1953-01-10#004410124-000513,NaT,NaT
6253,нивл,аппарат искусственной вентиляции легких для пр...,004210124-0000126,выдано пациенту,расторгуева,елена,ивановна,1946-06-16,2023-07-15,NaT,расторгуева#1946-06-16#004210124-0000126,NaT,NaT
6254,аспиратор,"отсасыватель хирургический электрический ""arme...",б/н,выдано пациенту,матюшин,сергей,олегович,2002-11-14,2023-07-17,NaT,матюшин#2002-11-14#б/н#отсасыватель хирургичес...,NaT,NaT
6255,пульсоксиметр,пульсоксиметр (аппарат мониторинга пациента) n...,004410124-0006370,выдано пациенту,матюшин,сергей,олегович,2002-11-14,2023-07-17,NaT,матюшин#2002-11-14#004410124-0006370,NaT,NaT


In [902]:
# df_google_vozvrat_sh.loc[df_google_vozvrat_sh['Дата возврата'].isna()]

<div class="alert alert-block alert-info"><font size=3>
Оставим только нужные столбцы по выдаче-возврату 

In [903]:
df_google_poluch_vozvrat = df_google[[
# 'Тип оборудования', 
    'Единое наименование', 
    'Инвентарный номер',
       # 'Тип движения ТСР', 
    'Фамилия', 'Имя', 'Отчество', 'Дата рождения',
       'Дата выдачи_x', 
           'Дата возврата_y',
    # 'Дата возврата_x', 
    'Ключ_пац_прибор', 
    # 'Дата выдачи_y',
]]
df_google_poluch_vozvrat

,Единое наименование,Инвентарный номер,Фамилия,Имя,Отчество,Дата рождения,Дата выдачи_x,Дата возврата_y,Ключ_пац_прибор
0,концентратор кислородный jay в следующем вариа...,004410124-0002849,степлякова,мария,григорьевна,1940-07-08,2021-04-24,2021-05-10,степлякова#1940-07-08#004410124-0002849
1,концентратор кислородный jay в следующем вариа...,004410124-0002849,селиверстов,аркадий,иванович,1946-03-25,2021-05-11,2021-09-19,селиверстов#1946-03-25#004410124-0002849
2,концентратор кислородный jay в следующем вариа...,004410124-0002849,никитина,татьяна,николаевна,1940-06-11,2021-09-24,2021-10-20,никитина#1940-06-11#004410124-0002849
3,концентратор кислородный jay в следующем вариа...,004410124-0002849,спирина,ираида,васильевна,1938-09-24,2021-10-21,2021-10-26,спирина#1938-09-24#004410124-0002849
4,концентратор кислородный jay в следующем вариа...,004410124-0002849,бадамшин,сергей,чингисович,1954-08-13,2021-10-27,NaT,бадамшин#1954-08-13#004410124-0002849
...,...,...,...,...,...,...,...,...,...
6252,"концентратор кислорода invacare, вариант испол...",004410124-000513,медведев,владимир,зенонович,1953-01-10,2023-07-14,NaT,медведев#1953-01-10#004410124-000513
6253,аппарат искусственной вентиляции легких для пр...,004210124-0000126,расторгуева,елена,ивановна,1946-06-16,2023-07-15,NaT,расторгуева#1946-06-16#004210124-0000126
6254,"отсасыватель хирургический электрический ""arme...",б/н,матюшин,сергей,олегович,2002-11-14,2023-07-17,NaT,матюшин#2002-11-14#б/н#отсасыватель хирургичес...
6255,пульсоксиметр (аппарат мониторинга пациента) n...,004410124-0006370,матюшин,сергей,олегович,2002-11-14,2023-07-17,NaT,матюшин#2002-11-14#004410124-0006370


In [916]:
# df_google_poluch_vozvrat.loc[df_google_poluch_vozvrat['Дата выдачи_x'] > df_google_poluch_vozvrat['Дата возврата_y']]

####
<div class="alert alert-block alert-success">
    <!-- -info-warning-success-danger>   -->
<font size=5>
<b>ИТОГО ролучили результат по ВСЕМ выдачам-получениям ПРИБОРОВ с инвентарными номервми  </b>    </div>
<hr>

In [917]:
df_google_poluch_vozvrat

,Единое наименование,Инвентарный номер,Фамилия,Имя,Отчество,Дата рождения,Дата выдачи_x,Дата возврата_y,Ключ_пац_прибор
0,концентратор кислородный jay в следующем вариа...,004410124-0002849,степлякова,мария,григорьевна,1940-07-08,2021-04-24,2021-05-10,степлякова#1940-07-08#004410124-0002849
1,концентратор кислородный jay в следующем вариа...,004410124-0002849,селиверстов,аркадий,иванович,1946-03-25,2021-05-11,2021-09-19,селиверстов#1946-03-25#004410124-0002849
2,концентратор кислородный jay в следующем вариа...,004410124-0002849,никитина,татьяна,николаевна,1940-06-11,2021-09-24,2021-10-20,никитина#1940-06-11#004410124-0002849
3,концентратор кислородный jay в следующем вариа...,004410124-0002849,спирина,ираида,васильевна,1938-09-24,2021-10-21,2021-10-26,спирина#1938-09-24#004410124-0002849
4,концентратор кислородный jay в следующем вариа...,004410124-0002849,бадамшин,сергей,чингисович,1954-08-13,2021-10-27,NaT,бадамшин#1954-08-13#004410124-0002849
...,...,...,...,...,...,...,...,...,...
6252,"концентратор кислорода invacare, вариант испол...",004410124-000513,медведев,владимир,зенонович,1953-01-10,2023-07-14,NaT,медведев#1953-01-10#004410124-000513
6253,аппарат искусственной вентиляции легких для пр...,004210124-0000126,расторгуева,елена,ивановна,1946-06-16,2023-07-15,NaT,расторгуева#1946-06-16#004210124-0000126
6254,"отсасыватель хирургический электрический ""arme...",б/н,матюшин,сергей,олегович,2002-11-14,2023-07-17,NaT,матюшин#2002-11-14#б/н#отсасыватель хирургичес...
6255,пульсоксиметр (аппарат мониторинга пациента) n...,004410124-0006370,матюшин,сергей,олегович,2002-11-14,2023-07-17,NaT,матюшин#2002-11-14#004410124-0006370


####
<div class="alert alert-block alert-info">
    <!-- -info-warning-success-danger>   -->
<font size=5>
<b>Соединим с выдачами из 1С  </b>    ------------------------------- ост. НАдо проверить задвоенность выдас и по 1с и по гугл. в приоритете 1с.

<div class="alert alert-block alert-info">
df из 1С   </div>

In [920]:
df_1c = df_poluch_vozvtat_not_nul_nomenklat.copy()
df_1c['Источник'] = '1c'

In [921]:
df_1c = df_1c[['Пациент', 'ДР', 'Номенклатура', 'Инвент_ном', 'Дата_выдачи',
       'Дата_возвр', 'Ключ_пац_прибор', 'Источник']]

In [922]:
df_1c

,Пациент,ДР,Номенклатура,Инвент_ном,Дата_выдачи,Дата_возвр,Ключ_пац_прибор,Источник
0,шваков макар олегович,2014-04-14,концентратор кислородный jay-5a.,004410124-0003072,2021-04-14,NaT,шваков#2014-04-14#004410124-0003072,1c
1,мартынов павел дмитриевич,2010-06-20,аппарат для удаления выделений из дыхательных ...,004410124-0002667,2021-03-17,NaT,мартынов#2010-06-20#004410124-0002667,1c
2,дарчиева кристина арсеновна,2010-03-15,аппарат искуственной вентиляции легких puritan...,004410124-0003302,2021-03-26,NaT,дарчиева#2010-03-15#004410124-0003302,1c
3,бабаев асланхон акмалхонович,2019-06-05,концентратор кислородный jay-5a.,004410124-0003073,2021-03-31,2021-07-19,бабаев#2019-06-05#004410124-0003073,1c
4,горячев владислав сергеевич,2006-12-28,аппарат нивл wm 100 prisma30st,004410124-001791,2021-04-15,NaT,горячев#2006-12-28#004410124-001791,1c
...,...,...,...,...,...,...,...,...
3038,коваленко матвей сергеевич,2020-02-10,аппарат искуственной вентиляции легких puritan...,004410124-0003331,2023-07-14,NaT,коваленко#2020-02-10#004410124-0003331,1c
3039,медведев владимир зенонович,1953-01-10,"концентратор кислорода invacare, вариант испол...",004410124-000513,2023-07-14,NaT,медведев#1953-01-10#004410124-000513,1c
3040,расторгуева елена ивановна,1964-06-16,аппарат искусственной вентиляции легких для пр...,004210124-0000126,2023-07-15,NaT,расторгуева#1964-06-16#004210124-0000126,1c
3041,агапов андрей васильевич,1962-03-10,аппарат для лечения нарушений дыхания во сне w...,004410124-0006265,2023-07-17,NaT,агапов#1962-03-10#004410124-0006265,1c


In [924]:
# ------------------ Новый ключ --------------
df_1c['Ключ_пац_прибор_дат_выдачи'] = df_1c['Ключ_пац_прибор'] + '#' + (df_1c['Дата_выдачи'].dt.date).map(str)
df_1c

,Пациент,ДР,Номенклатура,Инвент_ном,Дата_выдачи,Дата_возвр,Ключ_пац_прибор,Источник,Ключ_пац_прибор_дат_выдачи
0,шваков макар олегович,2014-04-14,концентратор кислородный jay-5a.,004410124-0003072,2021-04-14,NaT,шваков#2014-04-14#004410124-0003072,1c,шваков#2014-04-14#004410124-0003072#2021-04-14
1,мартынов павел дмитриевич,2010-06-20,аппарат для удаления выделений из дыхательных ...,004410124-0002667,2021-03-17,NaT,мартынов#2010-06-20#004410124-0002667,1c,мартынов#2010-06-20#004410124-0002667#2021-03-17
2,дарчиева кристина арсеновна,2010-03-15,аппарат искуственной вентиляции легких puritan...,004410124-0003302,2021-03-26,NaT,дарчиева#2010-03-15#004410124-0003302,1c,дарчиева#2010-03-15#004410124-0003302#2021-03-26
3,бабаев асланхон акмалхонович,2019-06-05,концентратор кислородный jay-5a.,004410124-0003073,2021-03-31,2021-07-19,бабаев#2019-06-05#004410124-0003073,1c,бабаев#2019-06-05#004410124-0003073#2021-03-31
4,горячев владислав сергеевич,2006-12-28,аппарат нивл wm 100 prisma30st,004410124-001791,2021-04-15,NaT,горячев#2006-12-28#004410124-001791,1c,горячев#2006-12-28#004410124-001791#2021-04-15
...,...,...,...,...,...,...,...,...,...
3038,коваленко матвей сергеевич,2020-02-10,аппарат искуственной вентиляции легких puritan...,004410124-0003331,2023-07-14,NaT,коваленко#2020-02-10#004410124-0003331,1c,коваленко#2020-02-10#004410124-0003331#2023-07-14
3039,медведев владимир зенонович,1953-01-10,"концентратор кислорода invacare, вариант испол...",004410124-000513,2023-07-14,NaT,медведев#1953-01-10#004410124-000513,1c,медведев#1953-01-10#004410124-000513#2023-07-14
3040,расторгуева елена ивановна,1964-06-16,аппарат искусственной вентиляции легких для пр...,004210124-0000126,2023-07-15,NaT,расторгуева#1964-06-16#004210124-0000126,1c,расторгуева#1964-06-16#004210124-0000126#2023-...
3041,агапов андрей васильевич,1962-03-10,аппарат для лечения нарушений дыхания во сне w...,004410124-0006265,2023-07-17,NaT,агапов#1962-03-10#004410124-0006265,1c,агапов#1962-03-10#004410124-0006265#2023-07-17


<div class="alert alert-block alert-info">
df из ГУГЛ   </div>

In [925]:
df_gle = df_google_poluch_vozvrat.copy()
df_gle['Источник'] = 'goole'
df_gle

,Единое наименование,Инвентарный номер,Фамилия,Имя,Отчество,Дата рождения,Дата выдачи_x,Дата возврата_y,Ключ_пац_прибор,Источник
0,концентратор кислородный jay в следующем вариа...,004410124-0002849,степлякова,мария,григорьевна,1940-07-08,2021-04-24,2021-05-10,степлякова#1940-07-08#004410124-0002849,goole
1,концентратор кислородный jay в следующем вариа...,004410124-0002849,селиверстов,аркадий,иванович,1946-03-25,2021-05-11,2021-09-19,селиверстов#1946-03-25#004410124-0002849,goole
2,концентратор кислородный jay в следующем вариа...,004410124-0002849,никитина,татьяна,николаевна,1940-06-11,2021-09-24,2021-10-20,никитина#1940-06-11#004410124-0002849,goole
3,концентратор кислородный jay в следующем вариа...,004410124-0002849,спирина,ираида,васильевна,1938-09-24,2021-10-21,2021-10-26,спирина#1938-09-24#004410124-0002849,goole
4,концентратор кислородный jay в следующем вариа...,004410124-0002849,бадамшин,сергей,чингисович,1954-08-13,2021-10-27,NaT,бадамшин#1954-08-13#004410124-0002849,goole
...,...,...,...,...,...,...,...,...,...,...
6252,"концентратор кислорода invacare, вариант испол...",004410124-000513,медведев,владимир,зенонович,1953-01-10,2023-07-14,NaT,медведев#1953-01-10#004410124-000513,goole
6253,аппарат искусственной вентиляции легких для пр...,004210124-0000126,расторгуева,елена,ивановна,1946-06-16,2023-07-15,NaT,расторгуева#1946-06-16#004210124-0000126,goole
6254,"отсасыватель хирургический электрический ""arme...",б/н,матюшин,сергей,олегович,2002-11-14,2023-07-17,NaT,матюшин#2002-11-14#б/н#отсасыватель хирургичес...,goole
6255,пульсоксиметр (аппарат мониторинга пациента) n...,004410124-0006370,матюшин,сергей,олегович,2002-11-14,2023-07-17,NaT,матюшин#2002-11-14#004410124-0006370,goole


<div class="alert alert-block alert-info">
приведем df из ГУГЛ к такому же виду как и 1С   </div>

In [926]:
#  переименуем столбцы
df_gle.rename(columns={'Дата рождения': 'ДР'}, inplace=True)
df_gle.rename(columns={'Единое наименование': 'Номенклатура'}, inplace=True)
df_gle.rename(columns={'Инвентарный номер': 'Инвент_ном'}, inplace=True)
df_gle.rename(columns={'Дата выдачи_x': 'Дата_выдачи'}, inplace=True)
df_gle.rename(columns={'Дата возврата_y': 'Дата_возвр'}, inplace=True)
df_gle

,Номенклатура,Инвент_ном,Фамилия,Имя,Отчество,ДР,Дата_выдачи,Дата_возвр,Ключ_пац_прибор,Источник
0,концентратор кислородный jay в следующем вариа...,004410124-0002849,степлякова,мария,григорьевна,1940-07-08,2021-04-24,2021-05-10,степлякова#1940-07-08#004410124-0002849,goole
1,концентратор кислородный jay в следующем вариа...,004410124-0002849,селиверстов,аркадий,иванович,1946-03-25,2021-05-11,2021-09-19,селиверстов#1946-03-25#004410124-0002849,goole
2,концентратор кислородный jay в следующем вариа...,004410124-0002849,никитина,татьяна,николаевна,1940-06-11,2021-09-24,2021-10-20,никитина#1940-06-11#004410124-0002849,goole
3,концентратор кислородный jay в следующем вариа...,004410124-0002849,спирина,ираида,васильевна,1938-09-24,2021-10-21,2021-10-26,спирина#1938-09-24#004410124-0002849,goole
4,концентратор кислородный jay в следующем вариа...,004410124-0002849,бадамшин,сергей,чингисович,1954-08-13,2021-10-27,NaT,бадамшин#1954-08-13#004410124-0002849,goole
...,...,...,...,...,...,...,...,...,...,...
6252,"концентратор кислорода invacare, вариант испол...",004410124-000513,медведев,владимир,зенонович,1953-01-10,2023-07-14,NaT,медведев#1953-01-10#004410124-000513,goole
6253,аппарат искусственной вентиляции легких для пр...,004210124-0000126,расторгуева,елена,ивановна,1946-06-16,2023-07-15,NaT,расторгуева#1946-06-16#004210124-0000126,goole
6254,"отсасыватель хирургический электрический ""arme...",б/н,матюшин,сергей,олегович,2002-11-14,2023-07-17,NaT,матюшин#2002-11-14#б/н#отсасыватель хирургичес...,goole
6255,пульсоксиметр (аппарат мониторинга пациента) n...,004410124-0006370,матюшин,сергей,олегович,2002-11-14,2023-07-17,NaT,матюшин#2002-11-14#004410124-0006370,goole


<div class="alert alert-block alert-info">
# -------------- создадим поле ПАЦИЕНТ -------  </div>

In [927]:
# --------- новый признак ПАЦИЕНТ (соединим фам им и отч) ------------
# data_df.loc[~data_df['Отчество'].isna(), 'Пациент' ] = data_df['Фамилия'] + ' ' +  data_df['Имя'] + ' ' + data_df['Отчество']
df_gle.loc[
      (~df_gle['Фамилия'].isna())
    & (~df_gle['Имя'].isna())
    & (~df_gle['Отчество'].isna())
                                     , 'Пациент' ] = df_gle['Фамилия'] + ' ' +  df_gle['Имя'] + ' ' + df_gle['Отчество']

df_gle.loc[
      (df_gle['Фамилия'].isna())
    & (~df_gle['Имя'].isna())
    & (~df_gle['Отчество'].isna())
                                     , 'Пациент' ] = '-' + ' ' +  df_gle['Имя'] + ' ' + df_gle['Отчество']
df_gle.loc[
      (~df_gle['Фамилия'].isna())
    & (df_gle['Имя'].isna())
    & (~df_gle['Отчество'].isna())
                                     , 'Пациент' ] = df_gle['Фамилия'] + ' ' + df_gle['Отчество']

df_gle.loc[
      (~df_gle['Фамилия'].isna())
    & (~df_gle['Имя'].isna())
    & (df_gle['Отчество'].isna())
                                     , 'Пациент' ] = df_gle['Фамилия'] + ' ' +  df_gle['Имя'] 

df_gle.loc[
      (~df_gle['Фамилия'].isna())
    & (df_gle['Имя'].isna())
    & (df_gle['Отчество'].isna())
                                     , 'Пациент' ] = df_gle['Фамилия'] 

df_gle.loc[
      (df_gle['Фамилия'].isna())
    & (df_gle['Имя'].isna())
    & (df_gle['Отчество'].isna())
                                     , 'Пациент' ] = '-'

df_gle

,Номенклатура,Инвент_ном,Фамилия,Имя,Отчество,ДР,Дата_выдачи,Дата_возвр,Ключ_пац_прибор,Источник,Пациент
0,концентратор кислородный jay в следующем вариа...,004410124-0002849,степлякова,мария,григорьевна,1940-07-08,2021-04-24,2021-05-10,степлякова#1940-07-08#004410124-0002849,goole,степлякова мария григорьевна
1,концентратор кислородный jay в следующем вариа...,004410124-0002849,селиверстов,аркадий,иванович,1946-03-25,2021-05-11,2021-09-19,селиверстов#1946-03-25#004410124-0002849,goole,селиверстов аркадий иванович
2,концентратор кислородный jay в следующем вариа...,004410124-0002849,никитина,татьяна,николаевна,1940-06-11,2021-09-24,2021-10-20,никитина#1940-06-11#004410124-0002849,goole,никитина татьяна николаевна
3,концентратор кислородный jay в следующем вариа...,004410124-0002849,спирина,ираида,васильевна,1938-09-24,2021-10-21,2021-10-26,спирина#1938-09-24#004410124-0002849,goole,спирина ираида васильевна
4,концентратор кислородный jay в следующем вариа...,004410124-0002849,бадамшин,сергей,чингисович,1954-08-13,2021-10-27,NaT,бадамшин#1954-08-13#004410124-0002849,goole,бадамшин сергей чингисович
...,...,...,...,...,...,...,...,...,...,...,...
6252,"концентратор кислорода invacare, вариант испол...",004410124-000513,медведев,владимир,зенонович,1953-01-10,2023-07-14,NaT,медведев#1953-01-10#004410124-000513,goole,медведев владимир зенонович
6253,аппарат искусственной вентиляции легких для пр...,004210124-0000126,расторгуева,елена,ивановна,1946-06-16,2023-07-15,NaT,расторгуева#1946-06-16#004210124-0000126,goole,расторгуева елена ивановна
6254,"отсасыватель хирургический электрический ""arme...",б/н,матюшин,сергей,олегович,2002-11-14,2023-07-17,NaT,матюшин#2002-11-14#б/н#отсасыватель хирургичес...,goole,матюшин сергей олегович
6255,пульсоксиметр (аппарат мониторинга пациента) n...,004410124-0006370,матюшин,сергей,олегович,2002-11-14,2023-07-17,NaT,матюшин#2002-11-14#004410124-0006370,goole,матюшин сергей олегович


In [928]:
# ------------------ Новый ключ --------------
# df_1c['Ключ_пац_прибор_дат_выдачи'] = df_1c['Ключ_пац_прибор'] + '#' + (df_1c['Дата_выдачи'].dt.date).map(str)
df_gle['Ключ_пац_прибор_дат_выдачи'] = df_gle['Ключ_пац_прибор'] + '#' + (df_gle['Дата_выдачи'].dt.date).map(str)
# df_gle

In [930]:
df_gle = df_gle[[    'Пациент', 'ДР', 'Номенклатура', 'Инвент_ном',
       'Дата_выдачи', 'Дата_возвр', 'Ключ_пац_прибор', 'Источник', 'Ключ_пац_прибор_дат_выдачи']]
df_gle

,Пациент,ДР,Номенклатура,Инвент_ном,Дата_выдачи,Дата_возвр,Ключ_пац_прибор,Источник,Ключ_пац_прибор_дат_выдачи
0,степлякова мария григорьевна,1940-07-08,концентратор кислородный jay в следующем вариа...,004410124-0002849,2021-04-24,2021-05-10,степлякова#1940-07-08#004410124-0002849,goole,степлякова#1940-07-08#004410124-0002849#2021-0...
1,селиверстов аркадий иванович,1946-03-25,концентратор кислородный jay в следующем вариа...,004410124-0002849,2021-05-11,2021-09-19,селиверстов#1946-03-25#004410124-0002849,goole,селиверстов#1946-03-25#004410124-0002849#2021-...
2,никитина татьяна николаевна,1940-06-11,концентратор кислородный jay в следующем вариа...,004410124-0002849,2021-09-24,2021-10-20,никитина#1940-06-11#004410124-0002849,goole,никитина#1940-06-11#004410124-0002849#2021-09-24
3,спирина ираида васильевна,1938-09-24,концентратор кислородный jay в следующем вариа...,004410124-0002849,2021-10-21,2021-10-26,спирина#1938-09-24#004410124-0002849,goole,спирина#1938-09-24#004410124-0002849#2021-10-21
4,бадамшин сергей чингисович,1954-08-13,концентратор кислородный jay в следующем вариа...,004410124-0002849,2021-10-27,NaT,бадамшин#1954-08-13#004410124-0002849,goole,бадамшин#1954-08-13#004410124-0002849#2021-10-27
...,...,...,...,...,...,...,...,...,...
6252,медведев владимир зенонович,1953-01-10,"концентратор кислорода invacare, вариант испол...",004410124-000513,2023-07-14,NaT,медведев#1953-01-10#004410124-000513,goole,медведев#1953-01-10#004410124-000513#2023-07-14
6253,расторгуева елена ивановна,1946-06-16,аппарат искусственной вентиляции легких для пр...,004210124-0000126,2023-07-15,NaT,расторгуева#1946-06-16#004210124-0000126,goole,расторгуева#1946-06-16#004210124-0000126#2023-...
6254,матюшин сергей олегович,2002-11-14,"отсасыватель хирургический электрический ""arme...",б/н,2023-07-17,NaT,матюшин#2002-11-14#б/н#отсасыватель хирургичес...,goole,матюшин#2002-11-14#б/н#отсасыватель хирургичес...
6255,матюшин сергей олегович,2002-11-14,пульсоксиметр (аппарат мониторинга пациента) n...,004410124-0006370,2023-07-17,NaT,матюшин#2002-11-14#004410124-0006370,goole,матюшин#2002-11-14#004410124-0006370#2023-07-17


####
<div class="alert alert-block alert-success">
    <!-- -info-warning-success-danger>   -->
<font size=5>
<b>Соединяем 1С и Гугл датафреймы  </b>    </div>
<hr>

In [931]:
df_1c = df_1c.drop_duplicates('Ключ_пац_прибор_дат_выдачи')
df_1c.shape

(3034, 9)

In [932]:
df_gle = df_gle.drop_duplicates('Ключ_пац_прибор_дат_выдачи')
df_gle.shape

(6257, 9)

In [933]:
df_unit = pd.concat([df_1c, df_gle])
df_unit

,Пациент,ДР,Номенклатура,Инвент_ном,Дата_выдачи,Дата_возвр,Ключ_пац_прибор,Источник,Ключ_пац_прибор_дат_выдачи
0,шваков макар олегович,2014-04-14,концентратор кислородный jay-5a.,004410124-0003072,2021-04-14,NaT,шваков#2014-04-14#004410124-0003072,1c,шваков#2014-04-14#004410124-0003072#2021-04-14
1,мартынов павел дмитриевич,2010-06-20,аппарат для удаления выделений из дыхательных ...,004410124-0002667,2021-03-17,NaT,мартынов#2010-06-20#004410124-0002667,1c,мартынов#2010-06-20#004410124-0002667#2021-03-17
2,дарчиева кристина арсеновна,2010-03-15,аппарат искуственной вентиляции легких puritan...,004410124-0003302,2021-03-26,NaT,дарчиева#2010-03-15#004410124-0003302,1c,дарчиева#2010-03-15#004410124-0003302#2021-03-26
3,бабаев асланхон акмалхонович,2019-06-05,концентратор кислородный jay-5a.,004410124-0003073,2021-03-31,2021-07-19,бабаев#2019-06-05#004410124-0003073,1c,бабаев#2019-06-05#004410124-0003073#2021-03-31
4,горячев владислав сергеевич,2006-12-28,аппарат нивл wm 100 prisma30st,004410124-001791,2021-04-15,NaT,горячев#2006-12-28#004410124-001791,1c,горячев#2006-12-28#004410124-001791#2021-04-15
...,...,...,...,...,...,...,...,...,...
6252,медведев владимир зенонович,1953-01-10,"концентратор кислорода invacare, вариант испол...",004410124-000513,2023-07-14,NaT,медведев#1953-01-10#004410124-000513,goole,медведев#1953-01-10#004410124-000513#2023-07-14
6253,расторгуева елена ивановна,1946-06-16,аппарат искусственной вентиляции легких для пр...,004210124-0000126,2023-07-15,NaT,расторгуева#1946-06-16#004210124-0000126,goole,расторгуева#1946-06-16#004210124-0000126#2023-...
6254,матюшин сергей олегович,2002-11-14,"отсасыватель хирургический электрический ""arme...",б/н,2023-07-17,NaT,матюшин#2002-11-14#б/н#отсасыватель хирургичес...,goole,матюшин#2002-11-14#б/н#отсасыватель хирургичес...
6255,матюшин сергей олегович,2002-11-14,пульсоксиметр (аппарат мониторинга пациента) n...,004410124-0006370,2023-07-17,NaT,матюшин#2002-11-14#004410124-0006370,goole,матюшин#2002-11-14#004410124-0006370#2023-07-17


In [937]:
df_unit[df_unit.duplicated('Ключ_пац_прибор_дат_выдачи', keep=False)].sort_values('Ключ_пац_прибор_дат_выдачи')
# df_unit.shape[0]

,Пациент,ДР,Номенклатура,Инвент_ном,Дата_выдачи,Дата_возвр,Ключ_пац_прибор,Источник,Ключ_пац_прибор_дат_выдачи
2072,абаева амина рустамовна,2019-07-13,пульсоксиметр pm-60,004210134-0001459,2023-03-02,NaT,абаева#2019-07-13#004210134-0001459,1c,абаева#2019-07-13#004210134-0001459#2023-03-02
5526,абаева амина рустамовна,2019-07-13,пульсоксиметр pm-60,004210134-0001459,2023-03-02,NaT,абаева#2019-07-13#004210134-0001459,goole,абаева#2019-07-13#004210134-0001459#2023-03-02
5524,абаева амина рустамовна,2019-07-13,концентратор кислородный jay-5a.,004410134-0081666,2023-03-02,NaT,абаева#2019-07-13#004410134-0081666,goole,абаева#2019-07-13#004410134-0081666#2023-03-02
2074,абаева амина рустамовна,2019-07-13,концентратор кислородный jay-5a.,004410134-0081666,2023-03-02,NaT,абаева#2019-07-13#004410134-0081666,1c,абаева#2019-07-13#004410134-0081666#2023-03-02
2073,абаева амина рустамовна,2019-07-13,"отсасыватель хирургический электрический ""arme...",004410134-0083445,2023-03-02,NaT,абаева#2019-07-13#004410134-0083445,1c,абаева#2019-07-13#004410134-0083445#2023-03-02
...,...,...,...,...,...,...,...,...,...
2568,яцуба любовь михайловна,1950-06-24,концентратор кислородный jay в следующем вариа...,004410124-0003227,2023-05-04,NaT,яцуба#1950-06-24#004410124-0003227,1c,яцуба#1950-06-24#004410124-0003227#2023-05-04
5646,яченков валерий григорьевич,1961-11-01,"prisma vent50, аппарат неинвазивной вентиляции...",004210124-0000027,2023-03-24,NaT,яченков#1961-11-01#004210124-0000027,goole,яченков#1961-11-01#004210124-0000027#2023-03-24
2076,яченков валерий григорьевич,1961-11-01,"prisma vent50, аппарат неинвазивной вентиляции...",004210124-0000027,2023-03-24,NaT,яченков#1961-11-01#004210124-0000027,1c,яченков#1961-11-01#004210124-0000027#2023-03-24
6201,яшин владимир александрович,1960-03-23,концентратор кислородный jay в следующем вариа...,004410134-0084434,2023-07-10,NaT,яшин#1960-03-23#004410134-0084434,goole,яшин#1960-03-23#004410134-0084434#2023-07-10


In [938]:
# -------------- УДАЛИМ ДУБЛИКАТЫ объедиенного ДАТАФРЕЙМА 1С и ГУГЛ
df_unit = df_unit.drop_duplicates('Ключ_пац_прибор_дат_выдачи')
df_unit

,Пациент,ДР,Номенклатура,Инвент_ном,Дата_выдачи,Дата_возвр,Ключ_пац_прибор,Источник,Ключ_пац_прибор_дат_выдачи
0,шваков макар олегович,2014-04-14,концентратор кислородный jay-5a.,004410124-0003072,2021-04-14,NaT,шваков#2014-04-14#004410124-0003072,1c,шваков#2014-04-14#004410124-0003072#2021-04-14
1,мартынов павел дмитриевич,2010-06-20,аппарат для удаления выделений из дыхательных ...,004410124-0002667,2021-03-17,NaT,мартынов#2010-06-20#004410124-0002667,1c,мартынов#2010-06-20#004410124-0002667#2021-03-17
2,дарчиева кристина арсеновна,2010-03-15,аппарат искуственной вентиляции легких puritan...,004410124-0003302,2021-03-26,NaT,дарчиева#2010-03-15#004410124-0003302,1c,дарчиева#2010-03-15#004410124-0003302#2021-03-26
3,бабаев асланхон акмалхонович,2019-06-05,концентратор кислородный jay-5a.,004410124-0003073,2021-03-31,2021-07-19,бабаев#2019-06-05#004410124-0003073,1c,бабаев#2019-06-05#004410124-0003073#2021-03-31
4,горячев владислав сергеевич,2006-12-28,аппарат нивл wm 100 prisma30st,004410124-001791,2021-04-15,NaT,горячев#2006-12-28#004410124-001791,1c,горячев#2006-12-28#004410124-001791#2021-04-15
...,...,...,...,...,...,...,...,...,...
6245,колунов алексей степанович,1935-12-05,аппарат для лечения нарушений дыхания во сне w...,004410124-001852,2023-07-13,NaT,колунов#1935-12-05#004410124-001852,goole,колунов#1935-12-05#004410124-001852#2023-07-13
6246,колунов алексей степанович,1935-12-05,концентратор кислородный jay в следующем вариа...,004410134-0081611,2023-07-13,NaT,колунов#1935-12-05#004410134-0081611,goole,колунов#1935-12-05#004410134-0081611#2023-07-13
6253,расторгуева елена ивановна,1946-06-16,аппарат искусственной вентиляции легких для пр...,004210124-0000126,2023-07-15,NaT,расторгуева#1946-06-16#004210124-0000126,goole,расторгуева#1946-06-16#004210124-0000126#2023-...
6254,матюшин сергей олегович,2002-11-14,"отсасыватель хирургический электрический ""arme...",б/н,2023-07-17,NaT,матюшин#2002-11-14#б/н#отсасыватель хирургичес...,goole,матюшин#2002-11-14#б/н#отсасыватель хирургичес...


In [234]:
# df_unit.loc[~df_unit['Инвент_ном'].str.contains('-') == True]

<div class="alert alert-block alert-info">
# ------------------ Новый ключ Ключ_3 ФИО НОМЕР (если цифровой) Дата выдачи Если не цифровой - то название ПРИБОРА--------------</div>

In [941]:
# # ------------------ Новый ключ Ключ_3 ФИО НОМЕР (если цифровой) Дата выдачи --------------
# df_unit.reset_index(drop= True , inplace= True )
# df_unit.loc[~df_unit['Инвент_ном'].str.contains('-') == True, 'Ключ_3' ] = df_unit['Пациент'].str.split(pat=' ', expand=True)[0] + '-' + df_unit['Инвент_ном'] + '-' + (df_unit['Дата_выдачи'].dt.date).map(str) + '-' + (df_unit['Дата_возвр'].dt.date).map(str) + df_unit['Номенклатура']
df_unit['Ключ_пац_прибор_дат_выдачи_возвр'] = df_unit['Ключ_пац_прибор_дат_выдачи'] + '#' + (df_unit['Дата_возвр'].dt.date).map(str)
df_unit

,Пациент,ДР,Номенклатура,Инвент_ном,Дата_выдачи,Дата_возвр,Ключ_пац_прибор,Источник,Ключ_пац_прибор_дат_выдачи,Ключ_пац_прибор_дат_выдачи_возвр
0,шваков макар олегович,2014-04-14,концентратор кислородный jay-5a.,004410124-0003072,2021-04-14,NaT,шваков#2014-04-14#004410124-0003072,1c,шваков#2014-04-14#004410124-0003072#2021-04-14,шваков#2014-04-14#004410124-0003072#2021-04-14...
1,мартынов павел дмитриевич,2010-06-20,аппарат для удаления выделений из дыхательных ...,004410124-0002667,2021-03-17,NaT,мартынов#2010-06-20#004410124-0002667,1c,мартынов#2010-06-20#004410124-0002667#2021-03-17,мартынов#2010-06-20#004410124-0002667#2021-03-...
2,дарчиева кристина арсеновна,2010-03-15,аппарат искуственной вентиляции легких puritan...,004410124-0003302,2021-03-26,NaT,дарчиева#2010-03-15#004410124-0003302,1c,дарчиева#2010-03-15#004410124-0003302#2021-03-26,дарчиева#2010-03-15#004410124-0003302#2021-03-...
3,бабаев асланхон акмалхонович,2019-06-05,концентратор кислородный jay-5a.,004410124-0003073,2021-03-31,2021-07-19,бабаев#2019-06-05#004410124-0003073,1c,бабаев#2019-06-05#004410124-0003073#2021-03-31,бабаев#2019-06-05#004410124-0003073#2021-03-31...
4,горячев владислав сергеевич,2006-12-28,аппарат нивл wm 100 prisma30st,004410124-001791,2021-04-15,NaT,горячев#2006-12-28#004410124-001791,1c,горячев#2006-12-28#004410124-001791#2021-04-15,горячев#2006-12-28#004410124-001791#2021-04-15...
...,...,...,...,...,...,...,...,...,...,...
6245,колунов алексей степанович,1935-12-05,аппарат для лечения нарушений дыхания во сне w...,004410124-001852,2023-07-13,NaT,колунов#1935-12-05#004410124-001852,goole,колунов#1935-12-05#004410124-001852#2023-07-13,колунов#1935-12-05#004410124-001852#2023-07-13...
6246,колунов алексей степанович,1935-12-05,концентратор кислородный jay в следующем вариа...,004410134-0081611,2023-07-13,NaT,колунов#1935-12-05#004410134-0081611,goole,колунов#1935-12-05#004410134-0081611#2023-07-13,колунов#1935-12-05#004410134-0081611#2023-07-1...
6253,расторгуева елена ивановна,1946-06-16,аппарат искусственной вентиляции легких для пр...,004210124-0000126,2023-07-15,NaT,расторгуева#1946-06-16#004210124-0000126,goole,расторгуева#1946-06-16#004210124-0000126#2023-...,расторгуева#1946-06-16#004210124-0000126#2023-...
6254,матюшин сергей олегович,2002-11-14,"отсасыватель хирургический электрический ""arme...",б/н,2023-07-17,NaT,матюшин#2002-11-14#б/н#отсасыватель хирургичес...,goole,матюшин#2002-11-14#б/н#отсасыватель хирургичес...,матюшин#2002-11-14#б/н#отсасыватель хирургичес...


In [943]:
# df_unit[df_unit.duplicated('Ключ_пац_прибор_дат_выдачи_возвр', keep=False)].sort_values('Ключ_пац_прибор_дат_выдачи_возвр')
# # df_unit

In [944]:
df_unit = df_unit.drop_duplicates('Ключ_пац_прибор_дат_выдачи_возвр')
df_unit

,Пациент,ДР,Номенклатура,Инвент_ном,Дата_выдачи,Дата_возвр,Ключ_пац_прибор,Источник,Ключ_пац_прибор_дат_выдачи,Ключ_пац_прибор_дат_выдачи_возвр
0,шваков макар олегович,2014-04-14,концентратор кислородный jay-5a.,004410124-0003072,2021-04-14,NaT,шваков#2014-04-14#004410124-0003072,1c,шваков#2014-04-14#004410124-0003072#2021-04-14,шваков#2014-04-14#004410124-0003072#2021-04-14...
1,мартынов павел дмитриевич,2010-06-20,аппарат для удаления выделений из дыхательных ...,004410124-0002667,2021-03-17,NaT,мартынов#2010-06-20#004410124-0002667,1c,мартынов#2010-06-20#004410124-0002667#2021-03-17,мартынов#2010-06-20#004410124-0002667#2021-03-...
2,дарчиева кристина арсеновна,2010-03-15,аппарат искуственной вентиляции легких puritan...,004410124-0003302,2021-03-26,NaT,дарчиева#2010-03-15#004410124-0003302,1c,дарчиева#2010-03-15#004410124-0003302#2021-03-26,дарчиева#2010-03-15#004410124-0003302#2021-03-...
3,бабаев асланхон акмалхонович,2019-06-05,концентратор кислородный jay-5a.,004410124-0003073,2021-03-31,2021-07-19,бабаев#2019-06-05#004410124-0003073,1c,бабаев#2019-06-05#004410124-0003073#2021-03-31,бабаев#2019-06-05#004410124-0003073#2021-03-31...
4,горячев владислав сергеевич,2006-12-28,аппарат нивл wm 100 prisma30st,004410124-001791,2021-04-15,NaT,горячев#2006-12-28#004410124-001791,1c,горячев#2006-12-28#004410124-001791#2021-04-15,горячев#2006-12-28#004410124-001791#2021-04-15...
...,...,...,...,...,...,...,...,...,...,...
6245,колунов алексей степанович,1935-12-05,аппарат для лечения нарушений дыхания во сне w...,004410124-001852,2023-07-13,NaT,колунов#1935-12-05#004410124-001852,goole,колунов#1935-12-05#004410124-001852#2023-07-13,колунов#1935-12-05#004410124-001852#2023-07-13...
6246,колунов алексей степанович,1935-12-05,концентратор кислородный jay в следующем вариа...,004410134-0081611,2023-07-13,NaT,колунов#1935-12-05#004410134-0081611,goole,колунов#1935-12-05#004410134-0081611#2023-07-13,колунов#1935-12-05#004410134-0081611#2023-07-1...
6253,расторгуева елена ивановна,1946-06-16,аппарат искусственной вентиляции легких для пр...,004210124-0000126,2023-07-15,NaT,расторгуева#1946-06-16#004210124-0000126,goole,расторгуева#1946-06-16#004210124-0000126#2023-...,расторгуева#1946-06-16#004210124-0000126#2023-...
6254,матюшин сергей олегович,2002-11-14,"отсасыватель хирургический электрический ""arme...",б/н,2023-07-17,NaT,матюшин#2002-11-14#б/н#отсасыватель хирургичес...,goole,матюшин#2002-11-14#б/н#отсасыватель хирургичес...,матюшин#2002-11-14#б/н#отсасыватель хирургичес...


<div class="alert alert-block alert-info">
<b>Выбросим записи. в которых не указана ДАТА ВЫДАЧИ!!!!</div>

In [946]:
df_unit = df_unit.loc[~df_unit['Дата_выдачи'].isna()]
df_unit

,Пациент,ДР,Номенклатура,Инвент_ном,Дата_выдачи,Дата_возвр,Ключ_пац_прибор,Источник,Ключ_пац_прибор_дат_выдачи,Ключ_пац_прибор_дат_выдачи_возвр
0,шваков макар олегович,2014-04-14,концентратор кислородный jay-5a.,004410124-0003072,2021-04-14,NaT,шваков#2014-04-14#004410124-0003072,1c,шваков#2014-04-14#004410124-0003072#2021-04-14,шваков#2014-04-14#004410124-0003072#2021-04-14...
1,мартынов павел дмитриевич,2010-06-20,аппарат для удаления выделений из дыхательных ...,004410124-0002667,2021-03-17,NaT,мартынов#2010-06-20#004410124-0002667,1c,мартынов#2010-06-20#004410124-0002667#2021-03-17,мартынов#2010-06-20#004410124-0002667#2021-03-...
2,дарчиева кристина арсеновна,2010-03-15,аппарат искуственной вентиляции легких puritan...,004410124-0003302,2021-03-26,NaT,дарчиева#2010-03-15#004410124-0003302,1c,дарчиева#2010-03-15#004410124-0003302#2021-03-26,дарчиева#2010-03-15#004410124-0003302#2021-03-...
3,бабаев асланхон акмалхонович,2019-06-05,концентратор кислородный jay-5a.,004410124-0003073,2021-03-31,2021-07-19,бабаев#2019-06-05#004410124-0003073,1c,бабаев#2019-06-05#004410124-0003073#2021-03-31,бабаев#2019-06-05#004410124-0003073#2021-03-31...
4,горячев владислав сергеевич,2006-12-28,аппарат нивл wm 100 prisma30st,004410124-001791,2021-04-15,NaT,горячев#2006-12-28#004410124-001791,1c,горячев#2006-12-28#004410124-001791#2021-04-15,горячев#2006-12-28#004410124-001791#2021-04-15...
...,...,...,...,...,...,...,...,...,...,...
6245,колунов алексей степанович,1935-12-05,аппарат для лечения нарушений дыхания во сне w...,004410124-001852,2023-07-13,NaT,колунов#1935-12-05#004410124-001852,goole,колунов#1935-12-05#004410124-001852#2023-07-13,колунов#1935-12-05#004410124-001852#2023-07-13...
6246,колунов алексей степанович,1935-12-05,концентратор кислородный jay в следующем вариа...,004410134-0081611,2023-07-13,NaT,колунов#1935-12-05#004410134-0081611,goole,колунов#1935-12-05#004410134-0081611#2023-07-13,колунов#1935-12-05#004410134-0081611#2023-07-1...
6253,расторгуева елена ивановна,1946-06-16,аппарат искусственной вентиляции легких для пр...,004210124-0000126,2023-07-15,NaT,расторгуева#1946-06-16#004210124-0000126,goole,расторгуева#1946-06-16#004210124-0000126#2023-...,расторгуева#1946-06-16#004210124-0000126#2023-...
6254,матюшин сергей олегович,2002-11-14,"отсасыватель хирургический электрический ""arme...",б/н,2023-07-17,NaT,матюшин#2002-11-14#б/н#отсасыватель хирургичес...,goole,матюшин#2002-11-14#б/н#отсасыватель хирургичес...,матюшин#2002-11-14#б/н#отсасыватель хирургичес...


<div class="alert alert-block alert-success g">
<b>
ИТОГОВЫЙ ДАТАФРЕЙМ С ДАННЫМИ!!!!</div>

In [947]:
df_unit

,Пациент,ДР,Номенклатура,Инвент_ном,Дата_выдачи,Дата_возвр,Ключ_пац_прибор,Источник,Ключ_пац_прибор_дат_выдачи,Ключ_пац_прибор_дат_выдачи_возвр
0,шваков макар олегович,2014-04-14,концентратор кислородный jay-5a.,004410124-0003072,2021-04-14,NaT,шваков#2014-04-14#004410124-0003072,1c,шваков#2014-04-14#004410124-0003072#2021-04-14,шваков#2014-04-14#004410124-0003072#2021-04-14...
1,мартынов павел дмитриевич,2010-06-20,аппарат для удаления выделений из дыхательных ...,004410124-0002667,2021-03-17,NaT,мартынов#2010-06-20#004410124-0002667,1c,мартынов#2010-06-20#004410124-0002667#2021-03-17,мартынов#2010-06-20#004410124-0002667#2021-03-...
2,дарчиева кристина арсеновна,2010-03-15,аппарат искуственной вентиляции легких puritan...,004410124-0003302,2021-03-26,NaT,дарчиева#2010-03-15#004410124-0003302,1c,дарчиева#2010-03-15#004410124-0003302#2021-03-26,дарчиева#2010-03-15#004410124-0003302#2021-03-...
3,бабаев асланхон акмалхонович,2019-06-05,концентратор кислородный jay-5a.,004410124-0003073,2021-03-31,2021-07-19,бабаев#2019-06-05#004410124-0003073,1c,бабаев#2019-06-05#004410124-0003073#2021-03-31,бабаев#2019-06-05#004410124-0003073#2021-03-31...
4,горячев владислав сергеевич,2006-12-28,аппарат нивл wm 100 prisma30st,004410124-001791,2021-04-15,NaT,горячев#2006-12-28#004410124-001791,1c,горячев#2006-12-28#004410124-001791#2021-04-15,горячев#2006-12-28#004410124-001791#2021-04-15...
...,...,...,...,...,...,...,...,...,...,...
6245,колунов алексей степанович,1935-12-05,аппарат для лечения нарушений дыхания во сне w...,004410124-001852,2023-07-13,NaT,колунов#1935-12-05#004410124-001852,goole,колунов#1935-12-05#004410124-001852#2023-07-13,колунов#1935-12-05#004410124-001852#2023-07-13...
6246,колунов алексей степанович,1935-12-05,концентратор кислородный jay в следующем вариа...,004410134-0081611,2023-07-13,NaT,колунов#1935-12-05#004410134-0081611,goole,колунов#1935-12-05#004410134-0081611#2023-07-13,колунов#1935-12-05#004410134-0081611#2023-07-1...
6253,расторгуева елена ивановна,1946-06-16,аппарат искусственной вентиляции легких для пр...,004210124-0000126,2023-07-15,NaT,расторгуева#1946-06-16#004210124-0000126,goole,расторгуева#1946-06-16#004210124-0000126#2023-...,расторгуева#1946-06-16#004210124-0000126#2023-...
6254,матюшин сергей олегович,2002-11-14,"отсасыватель хирургический электрический ""arme...",б/н,2023-07-17,NaT,матюшин#2002-11-14#б/н#отсасыватель хирургичес...,goole,матюшин#2002-11-14#б/н#отсасыватель хирургичес...,матюшин#2002-11-14#б/н#отсасыватель хирургичес...


In [240]:
# df_unit.to_excel('df_unit.xlsx')

<div class="alert alert-block alert-info">
<b>
    подсчитаем ВОЗРАСТ

<div class="alert alert-block alert-warning"><font size=3>
<b>
    ФУНКЦИЯ РАСЧЕТА ВОЗРАСТА 

In [948]:
# --------------- ФУНКЦИЯ РАСЧЕТА ВОЗРАСТА --------------------- на входе: Старшая дата - Младшая дата ---------
def calculate_age(x,y):
    age = (y.year - x.year - ((y.month, y.day) < (x.month, x.day)))
    return age

In [949]:
df_unit['Возраст'] = df_unit.apply( lambda x:  calculate_age(x['ДР'], x['Дата_выдачи']),axis=1)
df_unit['Дети'] = df_unit['Возраст'].apply(lambda x: 'Дети' if x <= 17 else 'Взрослые')
df_unit

,Пациент,ДР,Номенклатура,Инвент_ном,Дата_выдачи,Дата_возвр,Ключ_пац_прибор,Источник,Ключ_пац_прибор_дат_выдачи,Ключ_пац_прибор_дат_выдачи_возвр,Возраст,Дети
0,шваков макар олегович,2014-04-14,концентратор кислородный jay-5a.,004410124-0003072,2021-04-14,NaT,шваков#2014-04-14#004410124-0003072,1c,шваков#2014-04-14#004410124-0003072#2021-04-14,шваков#2014-04-14#004410124-0003072#2021-04-14...,7.0,Дети
1,мартынов павел дмитриевич,2010-06-20,аппарат для удаления выделений из дыхательных ...,004410124-0002667,2021-03-17,NaT,мартынов#2010-06-20#004410124-0002667,1c,мартынов#2010-06-20#004410124-0002667#2021-03-17,мартынов#2010-06-20#004410124-0002667#2021-03-...,10.0,Дети
2,дарчиева кристина арсеновна,2010-03-15,аппарат искуственной вентиляции легких puritan...,004410124-0003302,2021-03-26,NaT,дарчиева#2010-03-15#004410124-0003302,1c,дарчиева#2010-03-15#004410124-0003302#2021-03-26,дарчиева#2010-03-15#004410124-0003302#2021-03-...,11.0,Дети
3,бабаев асланхон акмалхонович,2019-06-05,концентратор кислородный jay-5a.,004410124-0003073,2021-03-31,2021-07-19,бабаев#2019-06-05#004410124-0003073,1c,бабаев#2019-06-05#004410124-0003073#2021-03-31,бабаев#2019-06-05#004410124-0003073#2021-03-31...,1.0,Дети
4,горячев владислав сергеевич,2006-12-28,аппарат нивл wm 100 prisma30st,004410124-001791,2021-04-15,NaT,горячев#2006-12-28#004410124-001791,1c,горячев#2006-12-28#004410124-001791#2021-04-15,горячев#2006-12-28#004410124-001791#2021-04-15...,14.0,Дети
...,...,...,...,...,...,...,...,...,...,...,...,...
6245,колунов алексей степанович,1935-12-05,аппарат для лечения нарушений дыхания во сне w...,004410124-001852,2023-07-13,NaT,колунов#1935-12-05#004410124-001852,goole,колунов#1935-12-05#004410124-001852#2023-07-13,колунов#1935-12-05#004410124-001852#2023-07-13...,87.0,Взрослые
6246,колунов алексей степанович,1935-12-05,концентратор кислородный jay в следующем вариа...,004410134-0081611,2023-07-13,NaT,колунов#1935-12-05#004410134-0081611,goole,колунов#1935-12-05#004410134-0081611#2023-07-13,колунов#1935-12-05#004410134-0081611#2023-07-1...,87.0,Взрослые
6253,расторгуева елена ивановна,1946-06-16,аппарат искусственной вентиляции легких для пр...,004210124-0000126,2023-07-15,NaT,расторгуева#1946-06-16#004210124-0000126,goole,расторгуева#1946-06-16#004210124-0000126#2023-...,расторгуева#1946-06-16#004210124-0000126#2023-...,77.0,Взрослые
6254,матюшин сергей олегович,2002-11-14,"отсасыватель хирургический электрический ""arme...",б/н,2023-07-17,NaT,матюшин#2002-11-14#б/н#отсасыватель хирургичес...,goole,матюшин#2002-11-14#б/н#отсасыватель хирургичес...,матюшин#2002-11-14#б/н#отсасыватель хирургичес...,20.0,Взрослые


####
<div class="alert alert-block alert-success">
    <!-- -info-warning-success-danger>   -->
<font size=5>
<b>функция расчета выданного оборудования в периоде c ОБОРУДОВАНИЕМ
    </b>    </div>
<hr>

In [950]:
# ------------------------------------------------- функция расчета выданного оборудования в периоде c ОБОРУДОВАНИЕМ----------------
def period_vidachi_equipment(df, dt_start, dt_finish, type_equipment, krome_text='#####'):
    type_equipment = type_equipment.lower()
    df['Номенклатура'] = df['Номенклатура'].str.lower()
    
    df = df.loc[
                (df['Номенклатура'].str.contains(type_equipment) == True)
                 & (df['Номенклатура'].str.contains(krome_text) != True)   ]
    
    
    df = df.loc[  (df['Дата_выдачи'] < finish ) 
            &
             ( (df['Дата_возвр'] >= start) | (df['Дата_возвр'].isna()))]
    
    # ------------------------------------------------------------------------------уникальные ПАЦИЕНТЫ -------     
    count_uniq_pacients = len(df.Пациент.unique()) 
    # ------------------------------------------------------------------------------уникальные Инвентарные номера -------     
    count_uniq_numbers = len(df.Инвент_ном.unique())
    # --------------------------------------------------------------------------количество выдач ЭТОГО типа ПРИБОРА ВСЕГО (по Инвентарному номеру) -------     
    used_oborud = len(df.Инвент_ном) 
    # ------------------------------------------------------------------------------количество ПОВТОРНЫХ выдач по Инвентарному номеру -------     
    duplicate_numbers = len(df.Инвент_ном) - len(df.Инвент_ном.unique())
 # ------------------------------------------------------------------------------перечень возможных наименований по части слова  -------     
    text_name_pik =  df.Номенклатура.unique()
    
    return df,  count_uniq_pacients,   count_uniq_numbers, used_oborud, duplicate_numbers, text_name_pik   

In [951]:
# ------------------------------------------------- функция расчета выданного оборудования в периоде c ОБОРУДОВАНИЕМ----------------
def period_vidachi_equipment_deti(df, dt_start, dt_finish, type_equipment, krome_text='#####'):
    type_equipment = type_equipment.lower()
    df['Номенклатура'] = df['Номенклатура'].str.lower()
    
    df = df.loc[
                (df['Номенклатура'].str.contains(type_equipment) == True)
                 & (df['Номенклатура'].str.contains(krome_text) != True)   ]
    
    
    df = df.loc[  
               (df['Дата_выдачи'] < finish ) 
            &  ( (df['Дата_возвр'] >= start) | (df['Дата_возвр'].isna()))
            &  (df['Дети'] == 'Дети')
               
               ]
    
    # ------------------------------------------------------------------------------уникальные ПАЦИЕНТЫ -------     
    count_uniq_pacients = len(df.Пациент.unique()) 
    # ------------------------------------------------------------------------------уникальные Инвентарные номера -------     
    count_uniq_numbers = len(df.Инвент_ном.unique())
    # --------------------------------------------------------------------------количество выдач ЭТОГО типа ПРИБОРА ВСЕГО (по Инвентарному номеру) -------     
    used_oborud = len(df.Инвент_ном) 
    # ------------------------------------------------------------------------------количество ПОВТОРНЫХ выдач по Инвентарному номеру -------     
    duplicate_numbers = len(df.Инвент_ном) - len(df.Инвент_ном.unique())
 # ------------------------------------------------------------------------------перечень возможных наименований по части слова  -------     
    text_name_pik =  df.Номенклатура.unique()
    
    return df,  count_uniq_pacients,   count_uniq_numbers, used_oborud, duplicate_numbers, text_name_pik   

####
<div class="alert alert-block alert-success">
    <!-- -info-warning-success-danger>   -->
<font size=5>
<b>Результат - подсчет в итоговом, объединенном из 1С и гугл датафрейме df_unit
    </b>    </div>
<hr>

In [957]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - нужно для обнуления ненужных слов в датафрейме'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'ventilogic'
# oborud = 'benne'
# krome = 'подставка-держатель'
    
df_data = df_unit

# df_ = period_vidachi_equipment_deti(df_data, start, finish, oborud, krome)
df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

 # Статистика выдачи оборудования РЦ в период с 01.07.2022 по 01.07.2023: 
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __ventilogic__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __59__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __82__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __99__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __17__ | 

* для получения датафрейма --> df_[0]  


****************************************************************************************************************************************************************************
аппарат для вентиляции легких ventilogic вариант исполнения ventilogic  plus с принадлежностями
аппарат для вентиляции легких ventilogic с принадлежностями, вариант исполнения ventilogic ls произв
аппарат искусственной вентиляции легких для проведения ивл/нивл (ventilogic ls)
аппарат для вентиляции легких ventilogic с принадлежностями, вариант исполнения ventilogic plus
аппарат для вентиляции легких ventilogic с принадлежностями, вариант исполнения  ventilogic ls
тележка для ventilogic ls
аппарат для вентиляции легких ventilogic с принадлежностями, вариант исполнения ventilogic ls
аппарат для вентиляции легких ventilogic вариант исполнения ventilogic plus с принадлежностями
аппарат для вентиляции легких ventilogic с принадлежностями, вариант исполнения ventilogic ls производства lowenstein medical technology gmbh + 

In [958]:
df_[0]

,Пациент,ДР,Номенклатура,Инвент_ном,Дата_выдачи,Дата_возвр,Ключ_пац_прибор,Источник,Ключ_пац_прибор_дат_выдачи,Ключ_пац_прибор_дат_выдачи_возвр,Возраст,Дети
268,бегинин алексей иванович,1977-08-22,аппарат для вентиляции легких ventilogic вариа...,004410124-000299,2021-12-23,2022-11-17,бегинин#1977-08-22#004410124-000299,1c,бегинин#1977-08-22#004410124-000299#2021-12-23,бегинин#1977-08-22#004410124-000299#2021-12-23...,44.0,Взрослые
325,свиридова мария александровна,2002-02-19,аппарат для вентиляции легких ventilogic с при...,004410124-0002214,2022-02-11,NaT,свиридова#2002-02-19#004410124-0002214,1c,свиридова#2002-02-19#004410124-0002214#2022-02-11,свиридова#2002-02-19#004410124-0002214#2022-02...,19.0,Взрослые
327,колпакова нина васильевна,1937-04-21,аппарат для вентиляции легких ventilogic с при...,004410124-0002243,2022-02-03,NaT,колпакова#1937-04-21#004410124-0002243,1c,колпакова#1937-04-21#004410124-0002243#2022-02-03,колпакова#1937-04-21#004410124-0002243#2022-02...,84.0,Взрослые
446,савченко тамара ивановна,1938-05-21,аппарат для вентиляции легких ventilogic с при...,004410124-0002225,2022-03-23,2023-01-10,савченко#1938-05-21#004410124-0002225,1c,савченко#1938-05-21#004410124-0002225#2022-03-23,савченко#1938-05-21#004410124-0002225#2022-03-...,83.0,Взрослые
461,шереметьев олег витальевич,1952-10-13,аппарат для вентиляции легких ventilogic вариа...,004410124-000295,2022-03-25,2022-07-29,шереметьев#1952-10-13#004410124-000295,1c,шереметьев#1952-10-13#004410124-000295#2022-03-25,шереметьев#1952-10-13#004410124-000295#2022-03...,69.0,Взрослые
...,...,...,...,...,...,...,...,...,...,...,...,...
4203,сотникова полина алексеевна,2002-12-03,аппарат для вентиляции легких ventilogic вариа...,004410124-0002205,2022-05-12,NaT,сотникова#2002-12-03#004410124-0002205,goole,сотникова#2002-12-03#004410124-0002205#2022-05-12,сотникова#2002-12-03#004410124-0002205#2022-05...,19.0,Взрослые
5024,сотникова полина алексеевна,2002-12-03,аппарат для вентиляции легких ventilogic вариа...,004210124-0000078,2022-11-08,2023-01-19,сотникова#2002-12-03#004210124-0000078,goole,сотникова#2002-12-03#004210124-0000078#2022-11-08,сотникова#2002-12-03#004210124-0000078#2022-11...,19.0,Взрослые
5025,воронин владимир юрьевич,1957-02-08,аппарат для вентиляции легких ventilogic вариа...,004410124-0002202,2022-11-03,2022-12-19,воронин#1957-02-08#004410124-0002202,goole,воронин#1957-02-08#004410124-0002202#2022-11-03,воронин#1957-02-08#004410124-0002202#2022-11-0...,65.0,Взрослые
5495,хеджази людмила александровна,1950-06-20,аппарат искусственной вентиляции легких для пр...,004210124-0000081,2023-02-28,NaT,хеджази#1950-06-20#004210124-0000081,goole,хеджази#1950-06-20#004210124-0000081#2023-02-28,хеджази#1950-06-20#004210124-0000081#2023-02-2...,72.0,Взрослые


In [959]:
df_[0].loc[df_[0]['Пациент'] == 'сотникова полина алексеевна']

,Пациент,ДР,Номенклатура,Инвент_ном,Дата_выдачи,Дата_возвр,Ключ_пац_прибор,Источник,Ключ_пац_прибор_дат_выдачи,Ключ_пац_прибор_дат_выдачи_возвр,Возраст,Дети
606,сотникова полина алексеевна,2002-12-03,аппарат для вентиляции легких ventilogic с при...,004410124-0002205,2022-05-13,NaT,сотникова#2002-12-03#004410124-0002205,1c,сотникова#2002-12-03#004410124-0002205#2022-05-13,сотникова#2002-12-03#004410124-0002205#2022-05...,19.0,Взрослые
1127,сотникова полина алексеевна,2002-12-03,аппарат искусственной вентиляции легких для пр...,004210124-0000078,2022-08-26,2023-01-19,сотникова#2002-12-03#004210124-0000078,1c,сотникова#2002-12-03#004210124-0000078#2022-08-26,сотникова#2002-12-03#004210124-0000078#2022-08...,19.0,Взрослые
1128,сотникова полина алексеевна,2002-12-03,аппарат для вентиляции легких ventilogic с при...,004410124-001396,2022-08-26,2022-11-08,сотникова#2002-12-03#004410124-001396,1c,сотникова#2002-12-03#004410124-001396#2022-08-26,сотникова#2002-12-03#004410124-001396#2022-08-...,19.0,Взрослые
1867,сотникова полина алексеевна,2002-12-03,аппарат искусственной вентиляции легких для пр...,004210124-0000078,2023-01-25,NaT,сотникова#2002-12-03#004210124-0000078,1c,сотникова#2002-12-03#004210124-0000078#2023-01-25,сотникова#2002-12-03#004210124-0000078#2023-01...,20.0,Взрослые
3204,сотникова полина алексеевна,2002-12-03,аппарат для вентиляции легких ventilogic с при...,004410124-0002239,2021-06-03,NaT,сотникова#2002-12-03#004410124-0002239,goole,сотникова#2002-12-03#004410124-0002239#2021-06-03,сотникова#2002-12-03#004410124-0002239#2021-06...,18.0,Взрослые
4203,сотникова полина алексеевна,2002-12-03,аппарат для вентиляции легких ventilogic вариа...,004410124-0002205,2022-05-12,NaT,сотникова#2002-12-03#004410124-0002205,goole,сотникова#2002-12-03#004410124-0002205#2022-05-12,сотникова#2002-12-03#004410124-0002205#2022-05...,19.0,Взрослые
5024,сотникова полина алексеевна,2002-12-03,аппарат для вентиляции легких ventilogic вариа...,004210124-0000078,2022-11-08,2023-01-19,сотникова#2002-12-03#004210124-0000078,goole,сотникова#2002-12-03#004210124-0000078#2022-11-08,сотникова#2002-12-03#004210124-0000078#2022-11...,19.0,Взрослые


In [956]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - нужно для обнуления ненужных слов в датафрейме'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'VIVO 40'
# krome = 'тележка'

df_data = df_unit

df_ = period_vidachi_equipment_deti(df_data, start, finish, oborud, krome)
# df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

 # Статистика выдачи оборудования РЦ в период с 01.07.2022 по 01.07.2023: 
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __VIVO 40__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __30__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __29__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __31__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __2__ | 

* для получения датафрейма --> df_[0]  


****************************************************************************************************************************************************************************
аппарат для обеспечения неинвазивной дыхательной поддержки vivo с принадлежностями (vivo 40)
****************************************************************************************************************************************************************************


In [219]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - нужно для обнуления ненужных слов в датафрейме'
start = '2020-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'VIVO 50'
# krome = 'тележка'

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

 # Статистика выдачи оборудования РЦ в период с 01.07.2020 по 01.07.2023: 
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __VIVO 50__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __3__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __2__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __3__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __1__ | 

* для получения датафрейма --> df_[0]  


****************************************************************************************************************************************************************************
аппарат для обеспечения дыхательной поддержки vivo 50 в комплекте с принадлежностями
****************************************************************************************************************************************************************************


In [221]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'CoughAssist'
# krome = 'тележка'

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

 # Статистика выдачи оборудования РЦ в период с 01.07.2022 по 01.07.2023: 
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __CoughAssist__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __310__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __267__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __367__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __100__ | 

* для получения датафрейма --> df_[0]  


****************************************************************************************************************************************************************************
аппарат для удаления выделений из дыхательных путей coughassist e70
аппарат для удаления выделений из дыхательных путей coughassist e70, тип 1 (coughassist e70, тип 1)
аппарат для удаления выделений из дыхательных путей coughassist e70 с принадлежнoстями
****************************************************************************************************************************************************************************


In [225]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'Prisma30ST'
# krome = 'тележка'

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)
# df_ = period_vidachi_equipment_deti(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

 # Статистика выдачи оборудования РЦ в период с 01.07.2022 по 01.07.2023: 
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __Prisma30ST__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __333__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __294__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __370__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __76__ | 

* для получения датафрейма --> df_[0]  


****************************************************************************************************************************************************************************
аппарат нивл wm 100 prisma30st
аппарат для лечения нарушений дыхания во сне wm 100 td, в варианте исполнения: prisma30st с принадл.
аппарат для лечения нарушений дыхания во сне wm 100 td, в варианте исполнения: prisma30st с принадлежностями.
аппарат для лечения нарушений дыхания во сне wm 100 td, в варианте исполнения: prisma30st с принадлежностями
****************************************************************************************************************************************************************************


In [64]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'VENT40'
# krome = 'тележка'

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

 # Статистика выдачи оборудования РЦ в период с 01.07.2022 по 01.07.2023: 
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __VENT40__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __70__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __50__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __71__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __21__ | 

* для получения датафрейма --> df_[0]  


****************************************************************************************************************************************************************************
аппарат искусственной вентиляции легких для проведения ивл/нивл (wt 100 td prisma vent40(нивл)
****************************************************************************************************************************************************************************


In [65]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'VENT50'
# krome = 'тележка'

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

 # Статистика выдачи оборудования РЦ в период с 01.07.2022 по 01.07.2023: 
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __VENT50__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __120__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __104__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __127__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __23__ | 

* для получения датафрейма --> df_[0]  


****************************************************************************************************************************************************************************
prisma vent50, аппарат неинвазивной вентиляции легких, с возможностью инвазивной вентиляции
аппарат искусственной авт. и вспомогательной вентиляции легких, тип wm 120td, prisma vent50
аппарат искусственной вентиляции легких для проведения ивл/нивл взрослым; (prisma vent50)
****************************************************************************************************************************************************************************


In [66]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'Bennett'
# krome = 'тележка'

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

 # Статистика выдачи оборудования РЦ в период с 01.07.2022 по 01.07.2023: 
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __Bennett__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __44__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __43__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __55__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __12__ | 

* для получения датафрейма --> df_[0]  


****************************************************************************************************************************************************************************
аппарат искуственной вентиляции легких puritan bennett 560, с принадлежностями
аппарат искусственной вентиляции легких puritan bennett 560, с принадлежностями
****************************************************************************************************************************************************************************


In [67]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'Super'
# krome = 'тележка'

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

 # Статистика выдачи оборудования РЦ в период с 01.07.2022 по 01.07.2023: 
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __Super__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __91__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __1__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __92__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __91__ | 

* для получения датафрейма --> df_[0]  


****************************************************************************************************************************************************************************
ингалятор компрессорный для аэрозольной терапии super-eco (model p0111em f400) с принадлежностями
****************************************************************************************************************************************************************************


In [68]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'Comfort Cough II'
# krome = 'тележка'

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

 # Статистика выдачи оборудования РЦ в период с 01.07.2022 по 01.07.2023: 
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __Comfort Cough II__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __45__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __37__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __49__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __12__ | 

* для получения датафрейма --> df_[0]  


****************************************************************************************************************************************************************************
инсуффлятор-аспиратор (comfort cough ii)
****************************************************************************************************************************************************************************


In [69]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'HORIZON P5'
# krome = 'тележка'

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

 # Статистика выдачи оборудования РЦ в период с 01.07.2022 по 01.07.2023: 
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __HORIZON P5__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __0__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __0__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __0__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __0__ | 

* для получения датафрейма --> df_[0]  


****************************************************************************************************************************************************************************
****************************************************************************************************************************************************************************


In [70]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'Invacare Perfecto2'
# krome = 'тележка'

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

 # Статистика выдачи оборудования РЦ в период с 01.07.2022 по 01.07.2023: 
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __Invacare Perfecto2__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __511__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __372__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __537__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __165__ | 

* для получения датафрейма --> df_[0]  


****************************************************************************************************************************************************************************
концентратор кислорода invacare, вариант исполнения invacare perfecto2 с принадлежностями
концентратор кислорода invacare, вариант исполнения invacare perfecto2 с принадлежностями (invacare perfecto2)
концентратор кислорода invacare, вариант исполнения invacare perfecto2 с принадлежностями 004410124-001604 19 bf-021258
****************************************************************************************************************************************************************************


In [71]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'Invacare XPO2'
# krome = 'тележка'

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

 # Статистика выдачи оборудования РЦ в период с 01.07.2022 по 01.07.2023: 
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __Invacare XPO2__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __109__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __112__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __132__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __20__ | 

* для получения датафрейма --> df_[0]  


****************************************************************************************************************************************************************************
концентратор кислорода invacare с принадлежностями вариант исполнения invacare xpo2
концентратор кислорода invacare с принадл., вариант исполнения invacare xpo2 (5 л.) 215690(портатив)
****************************************************************************************************************************************************************************


In [72]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'JAY-10'
# krome = 'тележка'

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

 # Статистика выдачи оборудования РЦ в период с 01.07.2022 по 01.07.2023: 
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __JAY-10__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __59__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __47__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __66__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __19__ | 

* для получения датафрейма --> df_[0]  


****************************************************************************************************************************************************************************
концентратор кислородный jay-10
концентратор кислородный jay в следующем варианте исполнения jay-10
концентратор кислородный jay вариант исполнения jay-10
****************************************************************************************************************************************************************************


In [73]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'JAY-5'
# krome = 'тележка'

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

 # Статистика выдачи оборудования РЦ в период с 01.07.2022 по 01.07.2023: 
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __JAY-5__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __877__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __635__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __982__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __347__ | 

* для получения датафрейма --> df_[0]  


****************************************************************************************************************************************************************************
концентратор кислородный jay-5a.
концентратор кислородный jay в следующем варианте исполнения jay-5a.
longfian jay-5
****************************************************************************************************************************************************************************


In [74]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'Матрас'
# krome = 'тележка'

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

 # Статистика выдачи оборудования РЦ в период с 01.07.2022 по 01.07.2023: 
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __Матрас__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __70__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __7__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __131__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __124__ | 

* для получения датафрейма --> df_[0]  


****************************************************************************************************************************************************************************
матрас противопролежневый (ячеистый).
матрас armed противопролежневый: ячеистый
матрас противопролежневый (ячеистый)
****************************************************************************************************************************************************************************


In [75]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'Amika'
# krome = 'тележка'

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

 # Статистика выдачи оборудования РЦ в период с 01.07.2022 по 01.07.2023: 
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __Amika__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __80__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __72__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __91__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __19__ | 

* для получения датафрейма --> df_[0]  


****************************************************************************************************************************************************************************
насос amika для подачи энтерального питания
насос amika для подачи энтерального питания_
****************************************************************************************************************************************************************************


In [76]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = '"armed", вариант исполнения: 7d'
# krome = 'тележка'

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

 # Статистика выдачи оборудования РЦ в период с 01.07.2022 по 01.07.2023: 
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __"armed", вариант исполнения: 7d__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __172__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __173__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __187__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __14__ | 

* для получения датафрейма --> df_[0]  


****************************************************************************************************************************************************************************
отсасыватель хирургический электрический "armed", вариант исполнения: 7d
****************************************************************************************************************************************************************************


In [77]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = '"armed", вариант исполнения: 7е-а'
# krome = 'тележка'

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

 # Статистика выдачи оборудования РЦ в период с 01.07.2022 по 01.07.2023: 
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __"armed", вариант исполнения: 7е-а__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __96__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __76__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __97__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __21__ | 

* для получения датафрейма --> df_[0]  


****************************************************************************************************************************************************************************
отсасыватель хирургический электрический "armed", вариант исполнения: 7е-а
****************************************************************************************************************************************************************************


In [78]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'подставка-держатель для аппарата ивл puritan bennet 560'
# krome = 'тележка'

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]cv
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

SyntaxError: invalid syntax (551136092.py, line 14)

In [ ]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'Nellcor Bedside SpO2'
# krome = 'тележка'

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

In [ ]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'Пульсоксиметр PM-60'
# krome = 'тележка'

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

In [ ]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'Пульсоксиметр серии MD300'
# krome = 'тележка'

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

In [ ]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'модель М70А'
# krome = 'тележка'

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

In [ ]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'модель М70В'
# krome = 'тележка'

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

In [ ]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = '"МЕГИДЕЗ"'
# krome = 'тележка'
# 'Рециркулятор бактерицидный для обеззараживания воздуха, на передвижной платформе "МЕГИДЕЗ" РБОВ 913-"МСК"'

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

In [ ]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'МЕГИДЕЗ'
# krome = 'тележка'
# Рециркулятор бактерицидный для обеззараживания воздуха, на передвижной платформе "МЕГИДЕЗ" РБОВ 913-"МСК"'
# Рециркулятор бактерицидный для обеззараживания воздуха, настенный "МЕГИДЕЗ" РБОВ 913-"МСК"

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

In [ ]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'PARI BOY Classic'
# krome = 'тележка'
# Система ингаляционная (ингалятор, небулайзер) PARI без подогрева PARI BOY Classic (тип 130)

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

In [ ]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'The Vest Airway Clearance System'
# krome = 'тележка'
# Система очистки дыхательных путей «The Vest Airway Clearance System, модель 105»

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

In [ ]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'Yang Kun YK-800'
# krome = 'тележка'
# Система очистки дыхательных путей «The Vest Airway Clearance System, модель 105»

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

In [ ]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'BIPAP'
# krome = 'тележка'
# BIPAP A40 Philips Respironics A40

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

In [ ]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'Тележка'
# krome = 'тележка'
# BIPAP A40 Philips Respironics A40

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

In [ ]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'Увлажнитель дыхательных смесей VADI VH-2000'
# krome = 'тележка'
# Увлажнитель дыхательных смесей MR810

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

In [ ]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'ТЕВЛАР'
# krome = 'тележка'
# Увлажнитель дыхательных смесей MR810

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

In [ ]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'Штатив для длительных инфузионных вливаний'
# krome = 'тележка'
# Увлажнитель дыхательных смесей MR810

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

In [ ]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'ВРАР System'
# krome = 'тележка'
# ИВЛ ВРАР System, вариант исполнения Т-30Т

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

In [ ]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'VIVO30'
# krome = 'тележка'
# ИВЛ ВРАР System, вариант исполнения Т-30Т

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

In [ ]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'Mark'
# krome = 'тележка'
# Mark 5 Nuvo Lite

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

In [ ]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'enteroport plus'
# krome = 'amika'
# Mark 5 Nuvo Lite

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

In [ ]:
# -----------выдано в 2021 году. Сколко уникальны пациентов
krome = '@не удалять - обязательно наличие для обнуления ненужных слов в датафрейме - КОСТЫЛЬ'
start = '2022-07-01'
finish = '2023-07-01'

# oborud = ''
oborud = 'Пульсоксиметр'
# krome = 'amika'
# Mark 5 Nuvo Lite

df_data = df_unit

df_ = period_vidachi_equipment(df_data, start, finish, oborud, krome)

df_pacients = df_[1]
df_oborud = df_[2]
df_used_oboruds = df_[3]
df_duplicate_numbers = df_[4]
text_name_pik = df_[5]


a = f'Статистика выдачи оборудования РЦ в период с {start.split("-")[2]}.{start.split("-")[1]}.{start.split("-")[0]} по {finish.split("-")[2]}.{finish.split("-")[1]}.{finish.split("-")[0]}: '
display_markdown(f''' # {a}
| Параметр | Количество| 
|---:|:-|
| В названии прибра присутствует текст | __{oborud}__ | 
| Количество уникальных пациентов, получивших оборудование в рассм.период| __{df_pacients}__ | 
| Количество уникальных ПРИБОРОВ (по инвент.номеру), выданных в рассм.период| __{df_oborud}__ | 
| Количество выдач ПРИБОРА (по инвент.номеру) в рассм.период (оборачива-емость)| __{df_used_oboruds}__ | 
| Количество повторных выдач ПРИБОРОВ (по инвент.номеру)| __{df_duplicate_numbers}__ | 

* для получения датафрейма --> df_[0]  
''', raw=True)
print(f'****************************************************************************************************************************************************************************')
for i in text_name_pik:
    print(i)
print(f'****************************************************************************************************************************************************************************')

In [ ]:
df_[0]

In [ ]:
df_[0]